In [1]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastCurrent, HistoricalAPI, HistoricalDaily, HistoricalHourly
import src
import pandas as pd
import datetime as dt

f = HistoricalAPI()
f.latitude, f.longitude = src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.RELATIVE_HUMIDITY_2M, HistoricalHourly.PRECIPITATION, HistoricalHourly.WIND_SPEED_10M,
            HistoricalHourly.WIND_DIRECTION_10M, HistoricalHourly.WIND_SPEED_100M, HistoricalHourly.WIND_DIRECTION_100M]
f.start_date = dt.datetime(2010, 1, 1)
f.end_date = dt.datetime(2024, 12, 31)
f.cell_selection = src.enums.CellSelection.NEAREST

response = f.request()
print(f.build_url())

https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&end_date=2024-12-31&hourly=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,wind_direction_10m,wind_speed_100m,wind_direction_100m&latitude=53.98352&longitude=-6.39139&start_date=2010-01-01&wind_speed_unit=ms


In [2]:
url=f.build_url()
conn = f._conn
src.weather.utils.ApiCounter.calculate_call_weight_from_url(url) 

273.9

In [3]:
src.GEO_COORDINATES.DUNDALK_IT.value

(53.98352, -6.39139)

In [4]:
parsedParams = src.weather.utils.ApiCounter.parse_url_params(url)
startDate = dt.datetime.strptime(parsedParams['start_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
endDate = dt.datetime.strptime(parsedParams['end_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['start_date'] = startDate.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['end_date'] = endDate.strftime('%Y-%m-%d %H:%M:%S')


```python
    pd.read_sql_query(f'''
                    SELECT * FROM hourly_historical_weather_data
                    WHERE time BETWEEN ? AND ?
                    AND {'1=0' if parsedParams.get('hourly') is None else 'parameter IN (' + ','.join("'"+str(x)+"'" for x in parsedParams['hourly']) + ')'}
                    AND latitude = ?
                    AND longitude = ?
                    ''', conn, params=[parsedParams['start_date'], parsedParams['end_date'], parsedParams['latitude'][0], parsedParams['longitude'][0]])
```

is equivalent to:

```python
    f = HistoricalAPI()
    f.latitude, f.longitude = src.GEO_COORDINATES.DUNDALK_IT.value
    f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
    f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.RELATIVE_HUMIDITY_2M, HistoricalHourly.PRECIPITATION, HistoricalHourly.WIND_SPEED_10M,
                HistoricalHourly.WIND_DIRECTION_10M, HistoricalHourly.WIND_SPEED_100M, HistoricalHourly.WIND_DIRECTION_100M]
    f.start_date = dt.datetime(2010, 1, 1)
    f.end_date = dt.datetime(2024, 12, 31)
    f.cell_selection = src.enums.CellSelection.NEAREST

    response = f.request()
```

yet the former takes only 6 seconds and the latter takes 45s


In [5]:
df = pd.read_sql_query(f'''
                  SELECT * FROM hourly_historical_weather_data
                  WHERE time BETWEEN ? AND ?
                  AND {'1=0' if parsedParams.get('hourly') is None else 'parameter IN (' + ','.join("'"+str(x)+"'" for x in parsedParams['hourly']) + ')'}
                  AND latitude = ?
                  AND longitude = ?
                  ''', conn, params=[parsedParams['start_date'], parsedParams['end_date'], parsedParams['latitude'][0], parsedParams['longitude'][0]])

In [6]:
from line_profiler import LineProfiler

lp = LineProfiler()
lp.add_function(f.request)  # Only profile this function
lp.add_function(f._convertToDictResponse)
lp.enable()
f.request()
lp.disable()
lp.print_stats(output_unit=1)


Timer unit: 1 s

Total time: 27.8794 s
File: c:\Users\local-admin\Desktop\Learning\17 - MSc Software Design with AI\06 Team Project\TurbineTeamProject\exploration\andrew prototyping\../..\src\weather\core.py
Function: request at line 64

Line #      Hits         Time  Per Hit   % Time  Line Contents
    64                                               def request(self)->list[dict]:
    65                                                   # build the url.
    66                                                   # parse the url into something that can be turned into a SQL query <- can do using my url parser
    67                                                   # query the data base.
    68                                                       # if it exists, return it. function terminates here.
    69                                                   # if it doesn't exist, request the data from the API
    70                                                   # parse the response to da

In [22]:

conn.execute('''
SELECT COUNT(*) - COUNT(DISTINCT time || latitude || longitude || utc_offset_seconds || 
                        timezone || timezone_abbreviation || elevation || parameter || value) 
FROM hourly_historical_weather_data;
'''
).fetchall()

[(0,)]

In [84]:
import cProfile

cProfile.run('f.request()', sort=-1)


reindexing tables
         1293096 function calls (1290981 primitive calls) in 74.258 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      657    0.001    0.000    0.005    0.000 <frozen abc>:117(__instancecheck__)
        5    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
       79    0.000    0.000    0.001    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        1    0.203    0.203   36.123   36.123 <string>:1(<module>)
       12    0.000    0.000    0.000    0.000 <string>:2(__hash__)
       12    0.000    0.000    0.000    0.000 <string>:2(__init__)
        2    0.000    0.000    0.000    0.000 __init__.py:164(match)
      157    0.000    0.000    0.001    0.000 __init__.py:174(search)
        4    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
        1    0.000    0.000    0.000    0.000 __init__.py:209(fi

In [41]:
out = '''
    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       12    0.000    0.000   43.712    3.643 base_events.py:1922(_run_once)
       12    0.040    0.003   43.709    3.642 selectors.py:319(select)
       12    0.055    0.005   41.320    3.443 selectors.py:313(_select)
       10   15.434    1.543   15.434    1.543 {method 'execute' of 'sqlite3.Connection' objects}
       12   12.150    1.013   13.897    1.158 {built-in method select.select}
        1    0.000    0.000   13.292   13.292 {built-in method builtins.exec}
        1    0.126    0.126   13.292   13.292 <string>:1(<module>)
        1    0.108    0.108   13.165   13.165 core.py:64(request)
        2    0.030    0.015   13.057    6.529 core.py:284(_convertToDictResponse)
        4    0.000    0.000   11.997    2.999 accessor.py:111(f)
        2    0.083    0.042    6.979    3.489 sql.py:434(read_sql_query)
        2    0.000    0.000    6.896    3.448 sql.py:2727(read_query)
        2    0.077    0.038    6.327    3.164 sql.py:2767(_fetchall_as_list)
        4    6.250    1.563    6.250    1.563 {method 'fetchall' of 'sqlite3.Cursor' objects}
        8    0.000    0.000    5.902    0.738 _decorators.py:325(wrapper)
        2    0.000    0.000    4.694    2.347 generic.py:2873(to_sql)
        2    1.180    0.590    4.694    2.347 sql.py:746(to_sql)
        2    0.043    0.022    3.465    1.733 sql.py:2773(to_sql)
        2    0.032    0.016    3.419    1.709 sql.py:1082(insert)
        1    0.000    0.000    3.032    3.032 sql.py:2545(_execute_insert)
        1    3.032    3.032    3.032    3.032 {method 'executemany' of 'sqlite3.Cursor' objects}
        4    0.021    0.005    1.961    0.490 accessors.py:126(_delegate_method)
        4    0.000    0.000    1.910    0.477 datetimes.py:276(strftime)
        4    0.000    0.000    1.908    0.477 datetimelike.py:1742(strftime)
        4    1.908    0.477    1.908    0.477 datetimes.py:744(_format_native_types)
       14    0.009    0.001    1.741    0.124 frame.py:5174(assign)
       92    0.002    0.000    1.719    0.019 managers.py:317(apply)
  222/210    0.000    0.000    1.656    0.008 common.py:372(apply_if_callable)
        2    0.053    0.027    1.648    0.824 core.py:322(<lambda>)
       18    0.000    0.000    1.499    0.083 generic.py:6463(astype)
       18    0.000    0.000    1.495    0.083 managers.py:421(astype)
       18    0.000    0.000    1.494    0.083 blocks.py:723(astype)
       18    0.000    0.000    1.493    0.083 astype.py:191(astype_array_safe)
       18    0.000    0.000    1.489    0.083 astype.py:157(astype_array)
       12    1.482    0.123    1.482    0.124 astype.py:56(_astype_nansafe)
        3    0.000    0.000    1.208    0.403 frame.py:2063(to_dict)
        3    0.000    0.000    1.208    0.403 to_dict.py:78(to_dict)
        9    0.463    0.051    1.203    0.134 to_dict.py:171(<genexpr>)
        1    0.000    0.000    0.755    0.755 frame.py:9492(pivot_table)
        1    0.023    0.023    0.755    0.755 pivot.py:61(pivot_table)
        9    0.000    0.000    0.750    0.083 generic.py:1420(aggregate)
  1051787    0.706    0.000    0.738    0.000 cast.py:183(maybe_box_native)
        1    0.000    0.000    0.732    0.732 pivot.py:118(__internal_pivot_table)
       10    0.000    0.000    0.710    0.071 ops.py:743(group_info)
       10    0.000    0.000    0.710    0.071 ops.py:758(_get_compressed_codes)
        9    0.000    0.000    0.706    0.078 apply.py:172(agg)
       57    0.014    0.000    0.695    0.012 algorithms.py:610(factorize)
       35    0.000    0.000    0.688    0.020 grouper.py:689(codes)
       19    0.000    0.000    0.688    0.036 grouper.py:777(_codes_and_uniques)
        9    0.000    0.000    0.673    0.075 groupby.py:1896(_agg_general)
        9    0.000    0.000    0.673    0.075 groupby.py:1957(_cython_agg_general)
        5    0.000    0.000    0.671    0.134 groupby.py:3337(first)
        9    0.000    0.000    0.670    0.074 groupby.py:1971(array_func)
        9    0.000    0.000    0.670    0.074 ops.py:812(_cython_operation)
        4    0.000    0.000    0.590    0.147 datetimes.py:673(to_datetime)
        5    0.003    0.001    0.573    0.115 ops.py:735(has_dropped_na)
        2    0.043    0.022    0.567    0.284 sql.py:194(_wrap_result)
        7    0.000    0.000    0.551    0.079 blocks.py:387(apply)
        1    0.000    0.000    0.551    0.551 apply.py:558(apply_str)
        1    0.000    0.000    0.550    0.550 apply.py:681(_apply_str)
        1    0.000    0.000    0.549    0.549 managers.py:1450(grouped_reduce)
       57    0.444    0.008    0.531    0.009 algorithms.py:548(factorize_array)
        2    0.185    0.093    0.522    0.261 sql.py:160(_convert_arrays_to_dataframe)
       14    0.000    0.000    0.443    0.032 ops.py:687(codes)
        5    0.416    0.083    0.416    0.083 {method 'commit' of 'sqlite3.Connection' objects}
       50    0.000    0.000    0.354    0.007 {built-in method builtins.next}
       21    0.000    0.000    0.354    0.017 contextlib.py:141(__exit__)
        2    0.000    0.000    0.354    0.177 sql.py:2656(run_transaction)
        4    0.000    0.000    0.348    0.087 datetimes.py:209(_maybe_cache)
      195    0.312    0.002    0.312    0.002 {method 'copy' of 'numpy.ndarray' objects}
    59/51    0.001    0.000    0.260    0.005 base.py:3820(get_indexer)
        4    0.000    0.000    0.256    0.064 frame.py:10813(merge)
  131/122    0.001    0.000    0.252    0.002 take.py:59(take_nd)
        2    0.000    0.000    0.247    0.124 construction.py:1006(convert_object_array)
        5    0.243    0.049    0.247    0.049 construction.py:1028(convert)
      120    0.196    0.002    0.246    0.002 take.py:120(_take_nd_ndarray)
        1    0.000    0.000    0.241    0.241 series.py:4620(map)
        1    0.000    0.000    0.240    0.240 base.py:891(_map_values)
        1    0.000    0.000    0.240    0.240 algorithms.py:1667(map_array)
        2    0.001    0.001    0.233    0.116 sql.py:1030(insert_data)
       24    0.001    0.000    0.232    0.010 managers.py:623(reindex_indexer)
        2    0.000    0.000    0.231    0.115 ops.py:607(get_iterator)
      8/4    0.010    0.001    0.229    0.057 merge.py:135(merge)
       34    0.000    0.000    0.228    0.007 blocks.py:1287(take_nd)
       62    0.001    0.000    0.227    0.004 managers.py:557(copy)
        4    0.000    0.000    0.222    0.056 merge.py:187(_cross_merge)
      101    0.001    0.000    0.219    0.002 blocks.py:790(copy)
       14    0.000    0.000    0.200    0.014 generic.py:4027(take)
       14    0.000    0.000    0.199    0.014 managers.py:869(take)
       27    0.197    0.007    0.198    0.007 base.py:3955(_get_indexer)
       43    0.001    0.000    0.196    0.005 generic.py:6662(copy)
        4    0.009    0.002    0.187    0.047 merge.py:882(get_result)
  122/120    0.000    0.000    0.180    0.001 missing.py:101(isna)
  122/120    0.001    0.000    0.179    0.001 missing.py:184(_isna)
        1    0.006    0.006    0.179    0.179 algorithms.py:307(unique)
       62    0.004    0.000    0.177    0.003 missing.py:261(_isna_array)
        1    0.159    0.159    0.173    0.173 algorithms.py:427(unique_with_mask)
       58    0.173    0.003    0.173    0.003 missing.py:305(_isna_string_dtype)
        8    0.000    0.000    0.159    0.020 common.py:62(new_method)
        8    0.000    0.000    0.159    0.020 arraylike.py:98(__add__)
        8    0.000    0.000    0.158    0.020 series.py:6133(_arith_method)
        8    0.000    0.000    0.158    0.020 base.py:1371(_arith_method)
        8    0.000    0.000    0.156    0.019 array_ops.py:240(arithmetic_op)
        8    0.000    0.000    0.156    0.019 array_ops.py:189(_na_arithmetic_op)
        8    0.000    0.000    0.156    0.019 apply.py:415(agg_dict_like)
        8    0.000    0.000    0.156    0.019 apply.py:1575(agg_or_apply_dict_like)
        8    0.155    0.019    0.155    0.019 {built-in method _operator.add}
        1    0.000    0.000    0.150    0.150 ops.py:622(_get_splitter)
       44    0.008    0.000    0.149    0.003 algorithms.py:1452(safe_sort)
        8    0.000    0.000    0.143    0.018 apply.py:425(compute_dict_like)
    22/10    0.000    0.000    0.136    0.014 generic.py:232(aggregate)
       10    0.001    0.000    0.136    0.014 generic.py:331(_aggregate_multiple_funcs)
       13    0.000    0.000    0.128    0.010 generic.py:4142(_take_with_is_copy)
        4    0.000    0.000    0.125    0.031 datetimes.py:314(_convert_listlike_datetimes)
        8    0.000    0.000    0.122    0.015 base.py:365(grouped_reduce)
        1    0.121    0.121    0.122    0.122 datetimes.py:456(_array_strptime_with_fallback)
       88    0.121    0.001    0.121    0.001 {method 'argsort' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.110    0.110 frame.py:9863(unstack)
        1    0.001    0.001    0.110    0.110 reshape.py:489(unstack)
        1    0.000    0.000    0.109    0.109 reshape.py:525(_unstack_frame)
       38    0.001    0.000    0.106    0.003 frame.py:4271(__setitem__)
       38    0.000    0.000    0.105    0.003 frame.py:4514(_set_item)
       58    0.000    0.000    0.101    0.002 algorithms.py:262(_get_hashtable_algo)
       20    0.001    0.000    0.100    0.005 frame.py:694(__init__)
       58    0.099    0.002    0.100    0.002 algorithms.py:280(_check_object_for_strings)
        1    0.000    0.000    0.099    0.099 reshape.py:110(__init__)
       15    0.001    0.000    0.097    0.006 construction.py:423(dict_to_mgr)
        4    0.000    0.000    0.094    0.023 merge.py:825(_reindex_and_concat)
       38    0.000    0.000    0.090    0.002 indexing.py:1176(__getitem__)
       15    0.000    0.000    0.088    0.006 construction.py:96(arrays_to_mgr)
        1    0.003    0.003    0.086    0.086 reshape.py:191(_make_selectors)
       15    0.000    0.000    0.084    0.006 managers.py:2121(create_block_manager_from_column_arrays)
       15    0.001    0.000    0.084    0.006 managers.py:2190(_form_blocks)
        4    0.000    0.000    0.083    0.021 expressions.py:226(evaluate)
        4    0.000    0.000    0.083    0.021 expressions.py:67(_evaluate_standard)
       16    0.070    0.004    0.083    0.005 managers.py:2246(_stack_arrays)
        2    0.000    0.000    0.082    0.041 core.py:223(_make_new_selfs)
        2    0.002    0.001    0.081    0.040 ops.py:1149(__iter__)
        1    0.000    0.000    0.079    0.079 ops.py:1162(_sorted_data)
        4    0.010    0.002    0.078    0.020 merge.py:1129(_get_join_info)
       53    0.000    0.000    0.076    0.001 frame.py:5242(_sanitize_column)
        4    0.000    0.000    0.073    0.018 indexing.py:1397(_getitem_axis)
        4    0.000    0.000    0.073    0.018 indexing.py:1205(_getbool_axis)
        4    0.003    0.001    0.069    0.017 merge.py:1119(_get_join_indexers)
       18    0.000    0.000    0.066    0.004 frame.py:12675(_reindex_for_setitem)
       83    0.001    0.000    0.066    0.001 frame.py:4062(__getitem__)
        4    0.002    0.000    0.065    0.016 merge.py:1692(get_join_indexers)
      109    0.059    0.001    0.063    0.001 base.py:2313(is_unique)
      177    0.061    0.000    0.061    0.000 {method 'astype' of 'numpy.ndarray' objects}
       18    0.001    0.000    0.058    0.003 managers.py:708(_slice_take_blocks_ax0)
        9    0.002    0.000    0.055    0.006 ops.py:771(ngroups)
154076/153749    0.044    0.000    0.054    0.000 {built-in method builtins.isinstance}
        9    0.000    0.000    0.053    0.006 ops.py:782(result_index)
        2    0.000    0.000    0.051    0.025 sql.py:2484(__init__)
        2    0.000    0.000    0.050    0.025 sql.py:933(__init__)
        2    0.000    0.000    0.050    0.025 sql.py:2556(_create_table_setup)
        2    0.000    0.000    0.050    0.025 sql.py:1243(_get_column_names_and_types)
        7    0.000    0.000    0.050    0.007 ops.py:776(reconstructed_codes)
        8    0.044    0.005    0.047    0.006 sql.py:2607(_sql_type_name)
       12    0.000    0.000    0.046    0.004 concat.py:157(concat)
        7    0.000    0.000    0.046    0.007 sorting.py:268(decons_obs_group_ids)
        7    0.046    0.007    0.046    0.007 sorting.py:246(_decons_group_index)
        9    0.000    0.000    0.046    0.005 ops.py:523(cython_operation)
  207/191    0.005    0.000    0.046    0.000 base.py:475(__new__)
        9    0.000    0.000    0.045    0.005 ops.py:310(_cython_op_ndim_compat)
        9    0.044    0.005    0.045    0.005 ops.py:354(_call_cython_op)
      710    0.044    0.000    0.044    0.000 {built-in method numpy.empty}
       12    0.000    0.000    0.044    0.004 concat.py:622(get_result)
        1    0.001    0.001    0.042    0.042 reshape.py:176(sorted_labels)
      630    0.007    0.000    0.040    0.000 generic.py:6301(__setattr__)
      153    0.038    0.000    0.038    0.000 {method 'take' of 'numpy.ndarray' objects}
    46/44    0.001    0.000    0.038    0.001 series.py:389(__init__)
    42/36    0.000    0.000    0.037    0.001 algorithms.py:1131(take)
       71    0.000    0.000    0.036    0.001 base.py:6394(_should_compare)
        2    0.000    0.000    0.036    0.018 sorting.py:214(get_compressed_ids)
      372    0.005    0.000    0.035    0.000 construction.py:517(sanitize_array)
       12    0.000    0.000    0.035    0.003 concat.py:94(concatenate_managers)
       76    0.033    0.000    0.034    0.000 base.py:2744(inferred_type)
        9    0.000    0.000    0.034    0.004 apply.py:1643(reconstruct_func)
        1    0.034    0.034    0.034    0.034 merge.py:1770(get_join_indexers_non_unique)
        8    0.000    0.000    0.034    0.004 apply.py:1738(normalize_keyword_aggregation)
       12    0.000    0.000    0.033    0.003 concat.py:202(_maybe_reindex_columns_na_proxy)
       38    0.000    0.000    0.032    0.001 frame.py:4481(_set_item_mgr)
        1    0.000    0.000    0.030    0.030 reshape.py:156(_indexer_and_to_sort)
        9    0.025    0.003    0.028    0.003 sorting.py:122(get_group_index)
        9    0.016    0.002    0.028    0.003 sorting.py:687(compress_group_index)
       16    0.000    0.000    0.027    0.002 multi.py:215(new_meth)
       16    0.001    0.000    0.026    0.002 multi.py:545(from_tuples)
       16    0.000    0.000    0.025    0.002 multi.py:474(from_arrays)
       38    0.001    0.000    0.025    0.001 categorical.py:371(__init__)
       16    0.000    0.000    0.023    0.001 categorical.py:3048(factorize_from_iterables)
       48    0.000    0.000    0.023    0.000 categorical.py:3069(<genexpr>)
       32    0.000    0.000    0.023    0.001 categorical.py:3005(factorize_from_iterable)
        1    0.022    0.022    0.022    0.022 {method 'searchsorted' of 'numpy.ndarray' objects}
       11    0.000    0.000    0.021    0.002 _mixins.py:157(take)
        9    0.001    0.000    0.020    0.002 frame.py:6239(reset_index)
       12    0.000    0.000    0.019    0.002 frame.py:4470(_iset_item_mgr)
       41    0.001    0.000    0.017    0.000 managers.py:1347(insert)
      103    0.002    0.000    0.017    0.000 base.py:674(_with_infer)
      259    0.012    0.000    0.017    0.000 cast.py:1157(maybe_infer_to_datetimelike)
      112    0.000    0.000    0.016    0.000 frame.py:4623(_clear_item_cache)
      279    0.016    0.000    0.016    0.000 {method 'clear' of 'dict' objects}
       17    0.000    0.000    0.016    0.001 base.py:1146(take)
        9    0.016    0.002    0.016    0.002 take.py:353(wrapper)
        2    0.014    0.007    0.014    0.007 sorting.py:644(get_group_index_sorter)
        1    0.000    0.000    0.014    0.014 base.py:269(join)
        1    0.000    0.000    0.014    0.014 base.py:4557(join)
       60    0.000    0.000    0.014    0.000 base.py:6324(_maybe_downcast_for_indexing)
        1    0.000    0.000    0.014    0.014 base.py:4983(_join_monotonic)
        1    0.013    0.013    0.013    0.013 base.py:400(_inner_indexer)
        1    0.012    0.012    0.012    0.012 multi.py:2083(remove_unused_levels)
      100    0.001    0.000    0.012    0.000 frame.py:4626(_get_item_cache)
        8    0.000    0.000    0.012    0.001 apply.py:504(wrap_results_dict_like)
        1    0.000    0.000    0.012    0.012 ops.py:946(_sorted_ids)
      118    0.001    0.000    0.012    0.000 frame.py:3983(_ixs)
       15    0.000    0.000    0.012    0.001 frame.py:5095(insert)
       11    0.011    0.001    0.011    0.001 {method 'cumsum' of 'numpy.ndarray' objects}
      392    0.001    0.000    0.011    0.000 base.py:7593(ensure_index)
      152    0.000    0.000    0.011    0.000 dtypes.py:220(__init__)
       32    0.000    0.000    0.011    0.000 indexing.py:1689(_getitem_tuple)
      272    0.011    0.000    0.011    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      152    0.000    0.000    0.011    0.000 dtypes.py:373(_finalize)
        4    0.000    0.000    0.011    0.003 merge.py:737(__init__)
       41    0.001    0.000    0.011    0.000 base.py:6956(insert)
        1    0.000    0.000    0.010    0.010 reshape.py:231(get_result)
       76    0.001    0.000    0.010    0.000 dtypes.py:550(validate_categories)
        2    0.000    0.000    0.010    0.005 frame.py:6538(dropna)
       24    0.009    0.000    0.009    0.000 datetimelike.py:2353(copy)
       32    0.000    0.000    0.009    0.000 indexing.py:1032(_getitem_lowerdim)
     4193    0.004    0.000    0.008    0.000 generic.py:42(_instancecheck)
       12    0.000    0.000    0.008    0.001 concat.py:699(new_axes)
      398    0.006    0.000    0.008    0.000 numeric.py:300(full)
       61    0.001    0.000    0.008    0.000 multi.py:325(__new__)
        1    0.000    0.000    0.008    0.008 reshape.py:378(new_index)
      101    0.001    0.000    0.008    0.000 generic.py:6284(__getattr__)
       34    0.000    0.000    0.008    0.000 indexing.py:1719(_getitem_axis)
       18    0.007    0.000    0.007    0.000 utils.py:239(maybe_convert_indices)
        1    0.000    0.000    0.007    0.007 ops.py:939(_sort_idx)
        5    0.000    0.000    0.007    0.001 generic.py:4796(_drop_axis)
        2    0.004    0.002    0.007    0.003 {method '__exit__' of 'sqlite3.Connection' objects}
       86    0.000    0.000    0.007    0.000 base.py:5552(equals)
        4    0.000    0.000    0.007    0.002 generic.py:1931(_drop_labels_or_levels)
       12    0.000    0.000    0.007    0.001 concat.py:717(_get_concat_axis)
       11    0.000    0.000    0.006    0.001 frame.py:9041(groupby)
       21    0.000    0.000    0.006    0.000 groupby.py:1296(__init__)
       59    0.000    0.000    0.006    0.000 base.py:6679(_maybe_cast_listlike_indexer)
        4    0.000    0.000    0.006    0.002 frame.py:5433(drop)
       42    0.006    0.000    0.006    0.000 {built-in method numpy.arange}
        4    0.000    0.000    0.006    0.002 generic.py:4757(drop)
        4    0.000    0.000    0.006    0.002 generic.py:4477(__delitem__)
6662/5002    0.004    0.000    0.006    0.000 {built-in method builtins.len}
       11    0.001    0.000    0.006    0.001 grouper.py:846(get_grouper)
        8    0.000    0.000    0.006    0.001 indexing.py:1093(_getitem_nested_tuple)
      238    0.003    0.000    0.006    0.000 base.py:5323(__contains__)
      116    0.001    0.000    0.006    0.000 frame.py:4608(_box_col_values)
       12    0.000    0.000    0.005    0.000 missing.py:380(notna)
        1    0.000    0.000    0.005    0.005 frame.py:7282(sort_index)
        1    0.000    0.000    0.005    0.005 generic.py:5290(sort_index)
        8    0.000    0.000    0.005    0.001 indexing.py:1696(_get_list_axis)
        8    0.001    0.000    0.005    0.001 concat.py:781(_make_concat_multiindex)
        2    0.000    0.000    0.005    0.002 frame.py:6997(sort_values)
       31    0.000    0.000    0.005    0.000 frame.py:1496(items)
      626    0.003    0.000    0.005    0.000 base.py:649(_simple_new)
        8    0.000    0.000    0.005    0.001 apply.py:636(normalize_dictlike_arg)
       20    0.000    0.000    0.005    0.000 base.py:6162(get_indexer_for)
     4193    0.003    0.000    0.005    0.000 generic.py:37(_check)
      141    0.002    0.000    0.005    0.000 base.py:842(_engine)
        4    0.000    0.000    0.005    0.001 groupby.py:853(ngroups)
       61    0.001    0.000    0.005    0.000 base.py:549(find)
       61    0.000    0.000    0.005    0.000 multi.py:1085(_set_codes)
      269    0.003    0.000    0.004    0.000 generic.py:6236(__finalize__)
       88    0.001    0.000    0.004    0.000 common.py:1596(pandas_dtype)
     1456    0.000    0.000    0.004    0.000 base.py:5144(_values)
        4    0.000    0.000    0.004    0.001 range.py:1148(take)
6210/6190    0.003    0.000    0.004    0.000 {built-in method builtins.getattr}
       42    0.000    0.000    0.004    0.000 base.py:5437(_can_hold_identifiers_and_holds_name)
       23    0.000    0.000    0.004    0.000 utils.py:187(validate_indices)
       14    0.000    0.000    0.004    0.000 generic.py:1870(_get_label_or_level_values)
      241    0.003    0.000    0.004    0.000 common.py:137(is_object_dtype)
        2    0.000    0.000    0.004    0.002 range.py:237(_data)
        5    0.000    0.000    0.004    0.001 base.py:7031(drop)
        1    0.004    0.004    0.004    0.004 {method 'put' of 'numpy.ndarray' objects}
       30    0.000    0.000    0.004    0.000 cast.py:1519(construct_1d_arraylike_from_scalar)
       18    0.004    0.000    0.004    0.000 {method 'fill' of 'numpy.ndarray' objects}
      191    0.000    0.000    0.004    0.000 multi.py:1102(<genexpr>)
      123    0.001    0.000    0.004    0.000 common.py:231(asarray_tuplesafe)
        2    0.000    0.000    0.004    0.002 frame.py:11603(any)
        2    0.000    0.000    0.004    0.002 generic.py:12169(_logical_func)
        2    0.000    0.000    0.004    0.002 sorting.py:301(lexsort_indexer)
        2    0.000    0.000    0.004    0.002 frame.py:11435(_reduce)
      116    0.003    0.000    0.003    0.000 managers.py:1012(iget)
        4    0.000    0.000    0.003    0.001 merge.py:1232(_get_merge_keys)
        2    0.000    0.000    0.003    0.002 core.py:225(<lambda>)
      152    0.000    0.000    0.003    0.000 {method 'any' of 'numpy.ndarray' objects}
      130    0.000    0.000    0.003    0.000 multi.py:4120(_coerce_indexer_frozen)
       39    0.000    0.000    0.003    0.000 {method 'max' of 'numpy.ndarray' objects}
      367    0.001    0.000    0.003    0.000 base.py:773(_view)
        1    0.003    0.003    0.003    0.003 merge.py:2398(_factorize_keys)
        3    0.003    0.001    0.003    0.001 sorting.py:165(maybe_lift)
       19    0.000    0.000    0.003    0.000 grouper.py:744(_group_index)
       39    0.000    0.000    0.003    0.000 _methods.py:42(_amax)
        8    0.000    0.000    0.003    0.000 base.py:3602(difference)
      168    0.003    0.000    0.003    0.000 cast.py:973(coerce_indexer_dtype)
      108    0.000    0.000    0.003    0.000 managers.py:583(copy_func)
      152    0.000    0.000    0.003    0.000 _methods.py:58(_any)
      104    0.000    0.000    0.003    0.000 generic.py:807(_set_axis)
       22    0.001    0.000    0.003    0.000 multi.py:3689(equals)
       15    0.000    0.000    0.003    0.000 construction.py:596(_homogenize)
      351    0.001    0.000    0.003    0.000 {built-in method builtins.any}
       12    0.003    0.000    0.003    0.000 base.py:2229(is_monotonic_increasing)
       67    0.001    0.000    0.003    0.000 cast.py:124(maybe_convert_platform)
      242    0.001    0.000    0.003    0.000 generic.py:339(_from_mgr)
       18    0.000    0.000    0.003    0.000 common.py:1277(is_extension_array_dtype)
       55    0.003    0.000    0.003    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
       13    0.000    0.000    0.003    0.000 events.py:86(_run)
       41    0.001    0.000    0.003    0.000 _function_base_impl.py:5518(insert)
      118    0.000    0.000    0.003    0.000 frame.py:678(_constructor_sliced_from_mgr)
      120    0.002    0.000    0.003    0.000 take.py:564(_take_preprocess_indexer_and_fill_value)
       13    0.000    0.000    0.003    0.000 {method 'run' of '_contextvars.Context' objects}
       11    0.000    0.000    0.003    0.000 {method 'min' of 'numpy.ndarray' objects}
       11    0.000    0.000    0.003    0.000 _methods.py:46(_amin)
      308    0.002    0.000    0.003    0.000 generic.py:278(__init__)
       74    0.000    0.000    0.002    0.000 managers.py:1782(_consolidate_inplace)
       21    0.000    0.000    0.002    0.000 multi.py:1248(copy)
       12    0.001    0.000    0.002    0.000 managers.py:1066(iset)
       14    0.000    0.000    0.002    0.000 ops.py:596(shape)
        2    0.000    0.000    0.002    0.001 frame.py:6488(isna)
        2    0.000    0.000    0.002    0.001 frame.py:1559(itertuples)
        4    0.001    0.000    0.002    0.001 datetimes.py:126(_guess_datetime_format_for_array)
       46    0.000    0.000    0.002    0.000 ops.py:598(<genexpr>)
        8    0.000    0.000    0.002    0.000 base.py:3664(_difference)
        2    0.000    0.000    0.002    0.001 base.py:176(isna)
       32    0.000    0.000    0.002    0.000 grouper.py:676(ngroups)
      202    0.001    0.000    0.002    0.000 _dtype.py:352(_name_get)
      170    0.001    0.000    0.002    0.000 base.py:5170(_get_engine_target)
      271    0.001    0.000    0.002    0.000 base.py:5373(__getitem__)
       16    0.000    0.000    0.002    0.000 {method 'extend' of 'list' objects}
       57    0.000    0.000    0.002    0.000 missing.py:466(array_equivalent)
        1    0.000    0.000    0.002    0.002 reshape.py:246(get_new_values)
       18    0.000    0.000    0.002    0.000 frame.py:1630(<genexpr>)
       61    0.001    0.000    0.002    0.000 multi.py:903(_set_levels)
        2    0.000    0.000    0.002    0.001 datetimes.py:821(date_range)
       14    0.000    0.000    0.002    0.000 generic.py:4159(xs)
       38    0.000    0.000    0.002    0.000 base.py:2817(hasnans)
      423    0.001    0.000    0.002    0.000 {built-in method builtins.all}
       81    0.000    0.000    0.002    0.000 base.py:6312(_index_as_unique)
        2    0.001    0.000    0.002    0.001 datetimes.py:397(_generate_range)
       47    0.000    0.000    0.002    0.000 generic.py:2077(__contains__)
        2    0.000    0.000    0.002    0.001 series.py:5786(notna)
        2    0.000    0.000    0.002    0.001 generic.py:8760(notna)
        4    0.001    0.000    0.002    0.000 merge.py:1002(_maybe_add_join_keys)
1053/1039    0.001    0.000    0.002    0.000 {built-in method numpy.asarray}
       35    0.002    0.000    0.002    0.000 numeric.py:2445(array_equal)
        8    0.000    0.000    0.002    0.000 series.py:6201(_construct_result)
        2    0.000    0.000    0.002    0.001 groupby.py:3201(min)
        4    0.000    0.000    0.002    0.000 sql.py:2668(execute)
        1    0.000    0.000    0.002    0.002 reshape.py:183(_make_sorted_values)
      261    0.001    0.000    0.002    0.000 base.py:7688(maybe_extract_name)
       12    0.000    0.000    0.002    0.000 concat.py:405(__init__)
       32    0.000    0.000    0.002    0.000 indexing.py:957(_validate_tuple_indexer)
        4    0.000    0.000    0.002    0.000 series.py:5773(isna)
      127    0.000    0.000    0.002    0.000 common.py:97(is_bool_indexer)
        4    0.000    0.000    0.002    0.000 generic.py:8693(isna)
        4    0.002    0.000    0.002    0.000 {method 'execute' of 'sqlite3.Cursor' objects}
        1    0.000    0.000    0.002    0.002 core.py:179(<lambda>)
       10    0.000    0.000    0.002    0.000 generic.py:6432(dtypes)
        4    0.000    0.000    0.002    0.000 merge.py:2697(_items_overlap_with_suffix)
        8    0.001    0.000    0.002    0.000 multi.py:1193(_engine)
        4    0.000    0.000    0.002    0.000 managers.py:1434(idelete)
        2    0.000    0.000    0.002    0.001 frame.py:5636(rename)
       15    0.000    0.000    0.002    0.000 multi.py:1316(view)
        2    0.000    0.000    0.002    0.001 generic.py:1070(_rename)
      134    0.001    0.000    0.002    0.000 base.py:3777(get_loc)
        4    0.000    0.000    0.002    0.000 base.py:3449(intersection)
        2    0.000    0.000    0.002    0.001 sql.py:139(_parse_date_columns)
       41    0.000    0.000    0.002    0.000 managers.py:1412(_insert_update_blklocs_and_blknos)
      657    0.001    0.000    0.002    0.000 <frozen abc>:117(__instancecheck__)
       10    0.000    0.000    0.002    0.000 datetimes.py:329(_from_sequence_not_strict)
      142    0.001    0.000    0.002    0.000 inference.py:273(is_dict_like)
        5    0.000    0.000    0.002    0.000 construction.py:237(ndarray_to_mgr)
      116    0.001    0.000    0.002    0.000 cast.py:551(maybe_promote)
      104    0.001    0.000    0.002    0.000 base.py:82(shape)
       18    0.001    0.000    0.002    0.000 generic.py:1771(_is_label_reference)
       64    0.000    0.000    0.001    0.000 indexing.py:1574(_validate_key)
       49    0.000    0.000    0.001    0.000 multi.py:844(levels)
        1    0.000    0.000    0.001    0.001 core.py:181(<lambda>)
       12    0.000    0.000    0.001    0.000 concat.py:707(_get_comb_axis)
      355    0.001    0.000    0.001    0.000 common.py:1434(_is_dtype_type)
       51    0.001    0.000    0.001    0.000 managers.py:1863(from_array)
        4    0.000    0.000    0.001    0.000 concat.py:777(_concat_indexes)
       94    0.000    0.000    0.001    0.000 frame.py:659(_constructor_from_mgr)
        4    0.000    0.000    0.001    0.000 managers.py:2259(_consolidate)
        4    0.000    0.000    0.001    0.000 base.py:5455(append)
       12    0.000    0.000    0.001    0.000 api.py:72(get_objs_combined_axis)
       12    0.000    0.000    0.001    0.000 managers.py:1796(concat_horizontal)
       94    0.000    0.000    0.001    0.000 base.py:1010(view)
        4    0.000    0.000    0.001    0.000 base.py:5491(_concat)
        5    0.000    0.000    0.001    0.000 generic.py:1565(__invert__)
       63    0.000    0.000    0.001    0.000 config.py:145(_get_option)
       41    0.000    0.000    0.001    0.000 managers.py:1402(_insert_update_mgr_locs)
      115    0.001    0.000    0.001    0.000 algorithms.py:184(_reconstruct_data)
      228    0.000    0.000    0.001    0.000 multi.py:830(__len__)
       38    0.000    0.000    0.001    0.000 dtypes.py:586(update_dtype)
        7    0.000    0.000    0.001    0.000 datetimes.py:320(__new__)
       12    0.000    0.000    0.001    0.000 api.py:120(_get_combined_index)
       82    0.001    0.000    0.001    0.000 _function_base_impl.py:5705(append)
       11    0.000    0.000    0.001    0.000 base.py:214(_obj_with_exclusions)
       10    0.000    0.000    0.001    0.000 generic.py:1953(_gotitem)
       38    0.000    0.000    0.001    0.000 base.py:2804(_isnan)
       58    0.000    0.000    0.001    0.000 algorithms.py:106(_ensure_data)
        1    0.000    0.000    0.001    0.001 generic.py:2002(_get_data_to_aggregate)
      108    0.000    0.000    0.001    0.000 base.py:782(_rename)
      191    0.000    0.000    0.001    0.000 multi.py:924(<genexpr>)
       36    0.000    0.000    0.001    0.000 dtypes.py:831(construct_from_string)
       41    0.000    0.000    0.001    0.000 numeric.py:1437(moveaxis)
      657    0.001    0.000    0.001    0.000 {built-in method _abc._abc_instancecheck}
       66    0.000    0.000    0.001    0.000 blocks.py:2703(new_block_2d)
      211    0.001    0.000    0.001    0.000 managers.py:180(blknos)
       42    0.000    0.000    0.001    0.000 warnings.py:133(filterwarnings)
        6    0.000    0.000    0.001    0.000 indexing.py:2632(check_bool_indexer)
        8    0.000    0.000    0.001    0.000 generic.py:4342(_getitem_slice)
        4    0.000    0.000    0.001    0.000 base.py:3535(_intersection)
      374    0.001    0.000    0.001    0.000 frozen.py:76(__getitem__)
      142    0.001    0.000    0.001    0.000 blocks.py:2645(maybe_coerce_values)
       20    0.000    0.000    0.001    0.000 utils.py:419(check_array_indexer)
      358    0.001    0.000    0.001    0.000 construction.py:481(ensure_wrapped_if_datetimelike)
       48    0.000    0.000    0.001    0.000 base.py:3035(unique)
        6    0.000    0.000    0.001    0.000 datetimelike.py:2307(factorize)
      200    0.001    0.000    0.001    0.000 warnings.py:484(__enter__)
     1000    0.001    0.000    0.001    0.000 {built-in method __new__ of type object at 0x00007FFA45D538F0}
        2    0.000    0.000    0.001    0.001 _ranges.py:24(generate_regular_range)
        6    0.000    0.000    0.001    0.000 base.py:1428(factorize)
      116    0.001    0.000    0.001    0.000 series.py:1471(_set_as_cached)
        3    0.000    0.000    0.001    0.000 {method 'sum' of 'numpy.ndarray' objects}
      639    0.001    0.000    0.001    0.000 inference.py:334(is_hashable)
      396    0.001    0.000    0.001    0.000 base.py:238(construct_from_string)
       80    0.000    0.000    0.001    0.000 common.py:1081(is_numeric_dtype)
        3    0.000    0.000    0.001    0.000 _methods.py:50(_sum)
        8    0.000    0.000    0.001    0.000 generic.py:4361(_slice)
       12    0.000    0.000    0.001    0.000 concat.py:494(_clean_keys_and_objs)
     1227    0.001    0.000    0.001    0.000 {built-in method builtins.hasattr}
        4    0.000    0.000    0.001    0.000 accessors.py:70(_get_values)
        2    0.000    0.000    0.001    0.000 sql.py:983(create)
      104    0.000    0.000    0.001    0.000 managers.py:236(set_axis)
        8    0.000    0.000    0.001    0.000 base.py:1028(nunique)
        5    0.000    0.000    0.001    0.000 indexing.py:2529(__setitem__)
        2    0.000    0.000    0.001    0.000 sql.py:969(exists)
        8    0.000    0.000    0.001    0.000 multi.py:2235(take)
        2    0.000    0.000    0.001    0.000 sql.py:2853(has_table)
       55    0.000    0.000    0.001    0.000 base.py:6415(_is_comparable_dtype)
        8    0.001    0.000    0.001    0.000 managers.py:2276(_merge_blocks)
       41    0.000    0.000    0.001    0.000 construction.py:769(_try_cast)
        1    0.000    0.000    0.001    0.001 series.py:1095(__getitem__)
       36    0.000    0.000    0.001    0.000 generic.py:1798(<genexpr>)
        4    0.000    0.000    0.001    0.000 base.py:3569(_intersection_via_get_indexer)
     1057    0.001    0.000    0.001    0.000 base.py:909(__len__)
       74    0.000    0.000    0.001    0.000 managers.py:1764(is_consolidated)
      343    0.001    0.000    0.001    0.000 construction.py:696(_sanitize_ndim)
      312    0.000    0.000    0.001    0.000 base.py:84(<genexpr>)
       24    0.000    0.000    0.001    0.000 datetimelike.py:373(__getitem__)
        5    0.000    0.000    0.001    0.000 series.py:1422(_set_value)
       88    0.000    0.000    0.001    0.000 cast.py:1580(construct_1d_object_array_from_listlike)
        5    0.000    0.000    0.001    0.000 series.py:1415(_set_values)
        8    0.000    0.000    0.001    0.000 missing.py:718(remove_na_arraylike)
      141    0.000    0.000    0.001    0.000 series.py:707(dtype)
       85    0.000    0.000    0.001    0.000 warnings.py:184(_add_filter)
       12    0.000    0.000    0.001    0.000 base.py:7750(_maybe_try_sort)
       80    0.001    0.000    0.001    0.000 blocks.py:2716(new_block)
      139    0.000    0.000    0.001    0.000 indexing.py:2765(check_dict_or_set_indexers)
        2    0.000    0.000    0.001    0.000 managers.py:1483(reduce)
        2    0.000    0.000    0.001    0.000 asyncio.py:200(_handle_events)
        5    0.000    0.000    0.001    0.000 construction.py:742(_get_axes)
        2    0.000    0.000    0.001    0.000 zmqstream.py:607(_handle_events)
       10    0.000    0.000    0.001    0.000 datetimes.py:2184(_sequence_to_dt64)
       30    0.000    0.000    0.001    0.000 series.py:664(_constructor_from_mgr)
       63    0.000    0.000    0.001    0.000 config.py:127(_get_single_key)
       40    0.001    0.000    0.001    0.000 {method 'match' of 're.Pattern' objects}
        5    0.000    0.000    0.001    0.000 iostream.py:118(_run_event_pipe_gc)
      113    0.000    0.000    0.001    0.000 common.py:1375(_is_dtype)
       74    0.000    0.000    0.001    0.000 managers.py:1772(_consolidate_check)
      173    0.001    0.000    0.001    0.000 base.py:122(_reset_cache)
       76    0.000    0.000    0.001    0.000 base.py:294(is_dtype)
      475    0.001    0.000    0.001    0.000 construction.py:416(extract_array)
       38    0.000    0.000    0.001    0.000 {method 'all' of 'numpy.ndarray' objects}
      411    0.001    0.000    0.001    0.000 common.py:1331(is_ea_or_datetimelike_dtype)
        1    0.000    0.000    0.001    0.001 base.py:6186(_get_indexer_strict)
        1    0.000    0.000    0.001    0.001 _parser.py:666(_parse)
        1    0.000    0.000    0.001    0.001 frame.py:3951(T)
      184    0.001    0.000    0.001    0.000 {built-in method numpy.array}
       17    0.000    0.000    0.001    0.000 fromnumeric.py:69(_wrapreduction)
       34    0.000    0.000    0.001    0.000 generic.py:1826(_check_label_or_level_ambiguity)
       27    0.000    0.000    0.001    0.000 blocks.py:274(make_block)
       38    0.000    0.000    0.001    0.000 dtypes.py:243(_from_values_or_dtype)
        1    0.000    0.000    0.001    0.001 frame.py:3767(transpose)
      141    0.001    0.000    0.001    0.000 managers.py:1993(dtype)
      104    0.000    0.000    0.001    0.000 base.py:86(_validate_set_axis)
       43    0.000    0.000    0.001    0.000 warnings.py:168(simplefilter)
       16    0.001    0.000    0.001    0.000 {built-in method numpy.zeros}
       92    0.000    0.000    0.001    0.000 common.py:1571(validate_all_hashable)
        6    0.000    0.000    0.001    0.000 fromnumeric.py:2513(all)
      547    0.001    0.000    0.001    0.000 __init__.py:34(using_copy_on_write)
        1    0.000    0.000    0.001    0.001 indexing.py:882(__setitem__)
       38    0.000    0.000    0.001    0.000 _methods.py:67(_all)
        4    0.000    0.000    0.001    0.000 managers.py:1216(_iset_split_block)
       32    0.000    0.000    0.001    0.000 generic.py:511(_validate_dtype)
        4    0.001    0.000    0.001    0.000 {built-in method _operator.invert}
      157    0.000    0.000    0.001    0.000 __init__.py:174(search)
        1    0.000    0.000    0.001    0.001 abstract_classes.py:112(build_url)
       33    0.000    0.000    0.001    0.000 common.py:81(get_op_result_name)
      195    0.000    0.000    0.001    0.000 series.py:831(_values)
       70    0.000    0.000    0.001    0.000 common.py:1198(is_bool_dtype)
       54    0.000    0.000    0.001    0.000 series.py:784(name)
        6    0.000    0.000    0.001    0.000 fromnumeric.py:89(_wrapreduction_any_all)
        1    0.000    0.000    0.001    0.001 ioloop.py:742(_run_callback)
      463    0.000    0.000    0.001    0.000 inference.py:300(<genexpr>)
      755    0.001    0.000    0.001    0.000 base.py:831(_reset_identity)
       24    0.000    0.000    0.001    0.000 blocks.py:324(slice_block_columns)
        3    0.000    0.000    0.001    0.000 datetimes.py:325(_from_sequence)
        1    0.000    0.000    0.001    0.001 iostream.py:616(_flush)
        9    0.000    0.000    0.001    0.000 missing.py:664(maybe_fill)
     1416    0.001    0.000    0.001    0.000 {built-in method builtins.issubclass}
      203    0.000    0.000    0.001    0.000 __init__.py:280(_compile)
      163    0.000    0.000    0.001    0.000 base.py:2776(_is_multi)
      140    0.000    0.000    0.001    0.000 blocks.py:2795(extend_blocks)
        1    0.000    0.000    0.001    0.001 core.py:180(<lambda>)
      198    0.000    0.000    0.001    0.000 base.py:609(_dtype_to_subclass)
       27    0.000    0.000    0.001    0.000 base.py:1697(_validate_names)
       12    0.000    0.000    0.001    0.000 fromnumeric.py:3068(prod)
        2    0.000    0.000    0.001    0.000 frame.py:1505(iterrows)
        6    0.000    0.000    0.001    0.000 series.py:1495(_maybe_update_cacher)
       82    0.000    0.000    0.001    0.000 numeric.py:1373(normalize_axis_tuple)
       35    0.000    0.000    0.001    0.000 blocks.py:1262(_slice)
        2    0.000    0.000    0.001    0.000 frame.py:12590(values)
       73    0.000    0.000    0.001    0.000 {built-in method builtins.sum}
        2    0.000    0.000    0.001    0.000 managers.py:1633(as_array)
       19    0.000    0.000    0.001    0.000 apply.py:1959(maybe_mangle_lambdas)
       45    0.000    0.000    0.001    0.000 multi.py:1574(_set_names)
      136    0.000    0.000    0.001    0.000 managers.py:913(__init__)
       68    0.000    0.000    0.001    0.000 range.py:201(_simple_new)
        2    0.000    0.000    0.001    0.000 managers.py:1707(_interleave)
       91    0.000    0.000    0.001    0.000 managers.py:948(from_blocks)
       41    0.000    0.000    0.001    0.000 managers.py:2311(_fast_count_smallints)
       24    0.000    0.000    0.001    0.000 _mixins.py:278(__getitem__)
       53    0.000    0.000    0.000    0.000 blocks.py:2811(ensure_block_shape)
      162    0.000    0.000    0.000    0.000 blocks.py:2674(get_block_type)
       61    0.000    0.000    0.000    0.000 enum.py:202(__get__)
        6    0.000    0.000    0.000    0.000 managers.py:372(setitem)
       14    0.000    0.000    0.000    0.000 datetimes.py:2428(maybe_convert_dtype)
       14    0.000    0.000    0.000    0.000 construction.py:638(_extract_index)
       53    0.000    0.000    0.000    0.000 frame.py:4585(_ensure_valid_index)
        1    0.000    0.000    0.000    0.000 cast.py:1200(maybe_cast_to_datetime)
        4    0.000    0.000    0.000    0.000 blocks.py:1935(delete)
       46    0.000    0.000    0.000    0.000 {built-in method builtins.sorted}
        1    0.000    0.000    0.000    0.000 core.py:182(<lambda>)
        4    0.000    0.000    0.000    0.000 datetimelike.py:1657(_groupby_op)
      325    0.000    0.000    0.000    0.000 construction.py:735(_sanitize_str_dtypes)
        2    0.000    0.000    0.000    0.000 groupby.py:3269(max)
      118    0.000    0.000    0.000    0.000 blocks.py:1253(iget)
       20    0.000    0.000    0.000    0.000 grouper.py:527(__init__)
      202    0.000    0.000    0.000    0.000 _dtype.py:336(_name_includes_bit_suffix)
       70    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
      182    0.000    0.000    0.000    0.000 generic.py:586(_get_axis)
       71    0.000    0.000    0.000    0.000 base.py:7723(_unpack_nested_dtype)
      261    0.000    0.000    0.000    0.000 flags.py:87(allows_duplicate_labels)
        5    0.000    0.000    0.000    0.000 base.py:6916(delete)
       30    0.000    0.000    0.000    0.000 common.py:536(is_string_dtype)
      200    0.000    0.000    0.000    0.000 warnings.py:505(__exit__)
      893    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
       12    0.000    0.000    0.000    0.000 managers.py:935(_verify_integrity)
       67    0.000    0.000    0.000    0.000 base.py:836(__iter__)
       82    0.000    0.000    0.000    0.000 fromnumeric.py:1846(ravel)
      141    0.000    0.000    0.000    0.000 base.py:456(_engine_type)
      308    0.000    0.000    0.000    0.000 flags.py:51(__init__)
        2    0.000    0.000    0.000    0.000 zmqstream.py:648(_handle_recv)
        2    0.000    0.000    0.000    0.000 series.py:607(_init_dict)
        1    0.000    0.000    0.000    0.000 session.py:754(send)
       61    0.000    0.000    0.000    0.000 enum.py:1286(value)
       69    0.000    0.000    0.000    0.000 common.py:568(require_length_match)
      175    0.000    0.000    0.000    0.000 generic.py:667(_info_axis)
        1    0.000    0.000    0.000    0.000 inspect.py:1358(getfullargspec)
       10    0.000    0.000    0.000    0.000 tasks.py:653(sleep)
        1    0.000    0.000    0.000    0.000 sorting.py:59(get_indexer_indexer)
       43    0.000    0.000    0.000    0.000 __init__.py:226(compile)
        8    0.000    0.000    0.000    0.000 multi.py:136(_codes_to_ints)
      135    0.000    0.000    0.000    0.000 series.py:734(name)
        9    0.000    0.000    0.000    0.000 groupby.py:1578(_wrap_aggregated_output)
      195    0.000    0.000    0.000    0.000 managers.py:2004(internal_values)
        5    0.000    0.000    0.000    0.000 series.py:6418(_reduce)
       59    0.000    0.000    0.000    0.000 base.py:3996(_check_indexing_method)
       58    0.000    0.000    0.000    0.000 algorithms.py:217(_ensure_arraylike)
      564    0.000    0.000    0.000    0.000 base.py:3809(<genexpr>)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:209(cell_selection)
       16    0.000    0.000    0.000    0.000 _asarray.py:27(require)
        3    0.000    0.000    0.000    0.000 datetimes.py:259(_box_as_indexlike)
        1    0.000    0.000    0.000    0.000 series.py:1216(_get_rows_with_mask)
        1    0.000    0.000    0.000    0.000 indexing.py:1785(_setitem_with_indexer)
       29    0.000    0.000    0.000    0.000 groupby.py:1357(__getattr__)
        5    0.000    0.000    0.000    0.000 futures.py:313(_set_result_unless_cancelled)
      198    0.000    0.000    0.000    0.000 base.py:591(_ensure_array)
       32    0.000    0.000    0.000    0.000 indexing.py:974(_is_nested_tuple_indexer)
       54    0.000    0.000    0.000    0.000 missing.py:564(_array_equivalent_object)
        1    0.000    0.000    0.000    0.000 base.py:5036(_wrap_joined_index)
        1    0.000    0.000    0.000    0.000 multi.py:2016(_sort_levels_monotonic)
      170    0.000    0.000    0.000    0.000 blocks.py:214(is_extension)
       36    0.000    0.000    0.000    0.000 managers.py:2264(<lambda>)
        4    0.000    0.000    0.000    0.000 nanops.py:482(nanany)
        4    0.000    0.000    0.000    0.000 accessor.py:220(__get__)
       35    0.000    0.000    0.000    0.000 api.py:386(default_index)
       79    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        2    0.000    0.000    0.000    0.000 blocks.py:398(reduce)
      356    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
       80    0.000    0.000    0.000    0.000 blocks.py:219(_can_consolidate)
        8    0.000    0.000    0.000    0.000 managers.py:1265(_iset_single)
       34    0.000    0.000    0.000    0.000 blocks.py:253(fill_value)
       18    0.000    0.000    0.000    0.000 managers.py:2320(_preprocess_slice_or_indexer)
      148    0.000    0.000    0.000    0.000 frame.py:1030(axes)
      541    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 managers.py:1954(get_rows_with_mask)
       32    0.000    0.000    0.000    0.000 indexing.py:1652(_is_scalar_access)
        9    0.000    0.000    0.000    0.000 socket.py:621(send)
        1    0.000    0.000    0.000    0.000 indexing.py:2179(_setitem_single_block)
       63    0.000    0.000    0.000    0.000 config.py:635(_get_root)
      148    0.000    0.000    0.000    0.000 common.py:1040(needs_i8_conversion)
        9    0.000    0.000    0.000    0.000 datetimelike.py:560(_validate_scalar)
       50    0.000    0.000    0.000    0.000 missing.py:728(is_valid_na_for_dtype)
        5    0.000    0.000    0.000    0.000 {method 'set_result' of '_asyncio.Future' objects}
      485    0.000    0.000    0.000    0.000 {built-in method _warnings._filters_mutated}
       10    0.000    0.000    0.000    0.000 datetimelike.py:2425(ensure_arraylike_for_datetimelike)
       85    0.000    0.000    0.000    0.000 {method 'remove' of 'list' objects}
      268    0.000    0.000    0.000    0.000 range.py:999(__len__)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        4    0.000    0.000    0.000    0.000 accessors.py:611(__new__)
        1    0.000    0.000    0.000    0.000 _parser.py:199(split)
       58    0.000    0.000    0.000    0.000 frame.py:1643(__len__)
       34    0.000    0.000    0.000    0.000 blocks.py:292(make_block_same_class)
        2    0.000    0.000    0.000    0.000 iostream.py:259(schedule)
        5    0.000    0.000    0.000    0.000 common.py:266(index_labels_to_array)
        2    0.000    0.000    0.000    0.000 zmqstream.py:580(_run_callback)
      170    0.000    0.000    0.000    0.000 managers.py:1392(<genexpr>)
      120    0.000    0.000    0.000    0.000 take.py:325(_get_take_nd_function)
        4    0.000    0.000    0.000    0.000 blocks.py:2244(get_values)
       12    0.000    0.000    0.000    0.000 base.py:3766(_convert_can_do_setop)
       20    0.000    0.000    0.000    0.000 common.py:1163(is_float_dtype)
       12    0.000    0.000    0.000    0.000 base.py:3439(_wrap_setop_result)
       50    0.000    0.000    0.000    0.000 indexing.py:1667(_validate_integer)
        6    0.000    0.000    0.000    0.000 blocks.py:1373(setitem)
      214    0.000    0.000    0.000    0.000 {method 'view' of 'numpy.ndarray' objects}
       16    0.000    0.000    0.000    0.000 apply.py:1799(_make_unique_kwarg_list)
        2    0.000    0.000    0.000    0.000 iostream.py:655(write)
        5    0.000    0.000    0.000    0.000 iostream.py:127(_event_pipe_gc)
        5    0.000    0.000    0.000    0.000 _function_base_impl.py:5335(delete)
        2    0.000    0.000    0.000    0.000 frame.py:11458(blk_func)
        3    0.000    0.000    0.000    0.000 traitlets.py:708(__set__)
        4    0.000    0.000    0.000    0.000 range.py:488(copy)
       25    0.000    0.000    0.000    0.000 fromnumeric.py:51(_wrapfunc)
      267    0.000    0.000    0.000    0.000 common.py:123(<lambda>)
        4    0.000    0.000    0.000    0.000 datetimes.py:674(astype)
       18    0.000    0.000    0.000    0.000 cast.py:775(infer_dtype_from_scalar)
        3    0.000    0.000    0.000    0.000 base.py:1922(rename)
      211    0.000    0.000    0.000    0.000 common.py:1590(<genexpr>)
        9    0.000    0.000    0.000    0.000 cast.py:259(maybe_downcast_to_dtype)
       29    0.000    0.000    0.000    0.000 common.py:103(_maybe_match_name)
        2    0.000    0.000    0.000    0.000 numeric.py:2315(isclose)
        5    0.000    0.000    0.000    0.000 fromnumeric.py:2781(max)
       38    0.000    0.000    0.000    0.000 categorical.py:862(codes)
       30    0.000    0.000    0.000    0.000 common.py:572(condition)
       11    0.000    0.000    0.000    0.000 base.py:1768(_set_names)
       20    0.000    0.000    0.000    0.000 grouper.py:1084(_convert_grouper)
        5    0.000    0.000    0.000    0.000 base_events.py:789(call_soon)
       63    0.000    0.000    0.000    0.000 range.py:376(dtype)
       36    0.000    0.000    0.000    0.000 dtypes.py:1305(construct_from_string)
        1    0.000    0.000    0.000    0.000 multi.py:1684(is_monotonic_increasing)
       10    0.000    0.000    0.000    0.000 managers.py:287(get_dtypes)
        3    0.000    0.000    0.000    0.000 base.py:1809(set_names)
      259    0.000    0.000    0.000    0.000 __init__.py:55(using_pyarrow_string_dtype)
       46    0.000    0.000    0.000    0.000 generic.py:592(_get_block_manager_axis)
        2    0.000    0.000    0.000    0.000 iostream.py:157(_handle_event)
       18    0.000    0.000    0.000    0.000 blocks.py:225(_consolidate_key)
        8    0.000    0.000    0.000    0.000 base.py:541(to_numpy)
       86    0.000    0.000    0.000    0.000 base.py:1765(_get_names)
       12    0.000    0.000    0.000    0.000 concat.py:545(_get_sample_object)
      200    0.000    0.000    0.000    0.000 warnings.py:458(__init__)
        6    0.000    0.000    0.000    0.000 base_events.py:747(call_later)
        2    0.000    0.000    0.000    0.000 series.py:6480(all)
       42    0.000    0.000    0.000    0.000 missing.py:673(na_value_for_dtype)
       36    0.000    0.000    0.000    0.000 dtypes.py:1054(construct_from_string)
       12    0.000    0.000    0.000    0.000 _parser.py:189(__next__)
        3    0.000    0.000    0.000    0.000 traitlets.py:689(set)
        1    0.000    0.000    0.000    0.000 session.py:690(serialize)
      2/1    0.000    0.000    0.000    0.000 inspect.py:2501(_signature_from_callable)
        1    0.000    0.000    0.000    0.000 reshape.py:325(get_new_columns)
        2    0.000    0.000    0.000    0.000 traitlets.py:3631(set)
      182    0.000    0.000    0.000    0.000 common.py:1399(_get_dtype)
       31    0.000    0.000    0.000    0.000 common.py:301(maybe_iterable_to_list)
        7    0.000    0.000    0.000    0.000 blocks.py:414(_split_op_result)
      972    0.000    0.000    0.000    0.000 typing.py:2187(cast)
        5    0.000    0.000    0.000    0.000 base_events.py:818(_call_soon)
       11    0.000    0.000    0.000    0.000 blocks.py:350(getitem_block_columns)
       12    0.000    0.000    0.000    0.000 concat.py:480(_get_ndims)
       25    0.000    0.000    0.000    0.000 series.py:914(__len__)
       10    0.000    0.000    0.000    0.000 generic.py:2082(empty)
      391    0.000    0.000    0.000    0.000 generic.py:572(_get_axis_number)
      250    0.000    0.000    0.000    0.000 common.py:152(cast_scalar_indexer)
        2    0.000    0.000    0.000    0.000 base.py:6520(_transform_index)
        8    0.000    0.000    0.000    0.000 generic.py:155(_wrap_agged_manager)
       24    0.000    0.000    0.000    0.000 range.py:483(_view)
      186    0.000    0.000    0.000    0.000 {built-in method numpy.asanyarray}
      200    0.000    0.000    0.000    0.000 base.py:974(dtype)
        2    0.000    0.000    0.000    0.000 _parser.py:875(_parse_numeric_token)
        1    0.000    0.000    0.000    0.000 base.py:4323(reindex)
       63    0.000    0.000    0.000    0.000 config.py:676(_translate_key)
        8    0.000    0.000    0.000    0.000 range.py:1032(_getitem_slice)
       62    0.000    0.000    0.000    0.000 inference.py:195(is_array_like)
        9    0.000    0.000    0.000    0.000 _shape_base_impl.py:1197(tile)
       12    0.000    0.000    0.000    0.000 _parser.py:77(get_token)
        2    0.000    0.000    0.000    0.000 series.py:6459(any)
        1    0.000    0.000    0.000    0.000 indexing.py:744(_get_setitem_indexer)
        6    0.000    0.000    0.000    0.000 generic.py:4883(_update_inplace)
       10    0.000    0.000    0.000    0.000 api.py:202(union_indexes)
        1    0.000    0.000    0.000    0.000 base.py:2107(droplevel)
       22    0.000    0.000    0.000    0.000 numerictypes.py:470(issubdtype)
       28    0.000    0.000    0.000    0.000 managers.py:1850(from_blocks)
       30    0.000    0.000    0.000    0.000 base.py:2794(_na_value)
        8    0.000    0.000    0.000    0.000 base.py:3675(_wrap_difference_result)
      193    0.000    0.000    0.000    0.000 {method 'search' of 're.Pattern' objects}
       12    0.000    0.000    0.000    0.000 datetimes.py:304(_simple_new)
       31    0.000    0.000    0.000    0.000 generic.py:4402(_check_setitem_copy)
      267    0.000    0.000    0.000    0.000 common.py:121(classes)
        2    0.000    0.000    0.000    0.000 generic.py:720(size)
        4    0.000    0.000    0.000    0.000 attrsettr.py:42(__getattr__)
        4    0.000    0.000    0.000    0.000 ipkernel.py:775(_clean_thread_parent_frames)
       21    0.000    0.000    0.000    0.000 contextlib.py:299(helper)
        1    0.000    0.000    0.000    0.000 inspect.py:2397(_signature_from_function)
        8    0.000    0.000    0.000    0.000 fromnumeric.py:2609(cumsum)
      343    0.000    0.000    0.000    0.000 construction.py:758(_maybe_repeat)
      108    0.000    0.000    0.000    0.000 base.py:791(is_)
      126    0.000    0.000    0.000    0.000 {method 'insert' of 'list' objects}
        2    0.000    0.000    0.000    0.000 iostream.py:577(_schedule_flush)
       24    0.000    0.000    0.000    0.000 merge.py:2684(_validate_operand)
       41    0.000    0.000    0.000    0.000 base.py:5295(_validate_fill_value)
      528    0.000    0.000    0.000    0.000 generic.py:405(flags)
       17    0.000    0.000    0.000    0.000 threading.py:1222(is_alive)
        6    0.000    0.000    0.000    0.000 base_events.py:771(call_at)
       16    0.000    0.000    0.000    0.000 generic.py:1740(_is_level_reference)
      414    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
      135    0.000    0.000    0.000    0.000 common.py:1270(is_1d_only_ea_dtype)
       23    0.000    0.000    0.000    0.000 frame.py:1047(shape)
      163    0.000    0.000    0.000    0.000 blocks.py:718(dtype)
        1    0.000    0.000    0.000    0.000 indexing.py:1452(_convert_to_indexer)
        3    0.000    0.000    0.000    0.000 traitlets.py:718(_validate)
        1    0.000    0.000    0.000    0.000 iostream.py:276(<lambda>)
        1    0.000    0.000    0.000    0.000 reshape.py:219(mask_all)
       40    0.000    0.000    0.000    0.000 common.py:311(is_null_slice)
      138    0.000    0.000    0.000    0.000 base.py:540(<genexpr>)
        1    0.000    0.000    0.000    0.000 cast.py:1240(_ensure_nanosecond_dtype)
        8    0.000    0.000    0.000    0.000 api.py:368(all_indexes_same)
        4    0.000    0.000    0.000    0.000 concat.py:52(concat_compat)
        1    0.000    0.000    0.000    0.000 iostream.py:278(_really_send)
       36    0.000    0.000    0.000    0.000 dtypes.py:2228(construct_from_string)
       16    0.000    0.000    0.000    0.000 multi.py:3801(_is_comparable_dtype)
        1    0.000    0.000    0.000    0.000 base.py:2159(_drop_level_numbers)
        9    0.000    0.000    0.000    0.000 datetimes.py:525(_unbox_scalar)
       36    0.000    0.000    0.000    0.000 dtypes.py:1835(construct_from_string)
        4    0.000    0.000    0.000    0.000 merge.py:1363(_maybe_coerce_merge_keys)
      198    0.000    0.000    0.000    0.000 managers.py:1837(__init__)
       92    0.000    0.000    0.000    0.000 common.py:529(is_string_or_object_np_dtype)
      400    0.000    0.000    0.000    0.000 multiarray.py:1089(copyto)
        2    0.000    0.000    0.000    0.000 {built-in method strptime}
        8    0.000    0.000    0.000    0.000 series.py:702(_can_hold_na)
       21    0.000    0.000    0.000    0.000 contextlib.py:132(__enter__)
        4    0.000    0.000    0.000    0.000 uuid.py:723(uuid4)
      261    0.000    0.000    0.000    0.000 generic.py:363(attrs)
        8    0.000    0.000    0.000    0.000 generic.py:160(_get_data_to_aggregate)
      320    0.000    0.000    0.000    0.000 base.py:1671(name)
       44    0.000    0.000    0.000    0.000 common.py:355(get_callable_name)
       25    0.000    0.000    0.000    0.000 base.py:74(__len__)
      202    0.000    0.000    0.000    0.000 _dtype.py:24(_kind_name)
        4    0.000    0.000    0.000    0.000 generic.py:1801(_is_label_or_level_reference)
      138    0.000    0.000    0.000    0.000 __init__.py:42(warn_copy_on_write)
       96    0.000    0.000    0.000    0.000 indexing.py:1181(<genexpr>)
        1    0.000    0.000    0.000    0.000 socket.py:698(send_multipart)
        2    0.000    0.000    0.000    0.000 _strptime.py:650(_strptime_datetime)
       88    0.000    0.000    0.000    0.000 common.py:131(<lambda>)
        1    0.000    0.000    0.000    0.000 series.py:6519(sum)
        9    0.000    0.000    0.000    0.000 apply.py:1524(__init__)
       36    0.000    0.000    0.000    0.000 string_.py:140(construct_from_string)
        4    0.000    0.000    0.000    0.000 session.py:92(json_packer)
        1    0.000    0.000    0.000    0.000 generic.py:12498(sum)
       32    0.000    0.000    0.000    0.000 indexing.py:935(_expand_ellipsis)
       12    0.000    0.000    0.000    0.000 _ufunc_config.py:410(__enter__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       15    0.000    0.000    0.000    0.000 sql.py:2460(_get_valid_sqlite_name)
        6    0.000    0.000    0.000    0.000 common.py:1342(is_complex_dtype)
        2    0.000    0.000    0.000    0.000 nanops.py:543(nanall)
        1    0.000    0.000    0.000    0.000 iostream.py:271(send_multipart)
        1    0.000    0.000    0.000    0.000 ops.py:1180(_chop)
      316    0.000    0.000    0.000    0.000 datetimes.py:551(dtype)
        8    0.000    0.000    0.000    0.000 shape_base.py:219(vstack)
       22    0.000    0.000    0.000    0.000 generic.py:2140(<genexpr>)
        1    0.000    0.000    0.000    0.000 generic.py:12459(_min_count_stat_function)
        1    0.000    0.000    0.000    0.000 putmask.py:129(setitem_datetimelike_compat)
      143    0.000    0.000    0.000    0.000 multi.py:1571(_get_names)
       36    0.000    0.000    0.000    0.000 dtypes.py:342(construct_from_string)
      342    0.000    0.000    0.000    0.000 multi.py:1081(codes)
        4    0.000    0.000    0.000    0.000 attrsettr.py:65(_get_attr_opt)
        2    0.000    0.000    0.000    0.000 base.py:3238(union)
        1    0.000    0.000    0.000    0.000 cast.py:762(infer_dtype_from)
       54    0.000    0.000    0.000    0.000 {built-in method numpy.datetime_data}
        8    0.000    0.000    0.000    0.000 base.py:3071(drop_duplicates)
        2    0.000    0.000    0.000    0.000 zmqstream.py:687(_rebuild_io_state)
        4    0.000    0.000    0.000    0.000 datetimes.py:149(should_cache)
        4    0.000    0.000    0.000    0.000 __init__.py:183(dumps)
       30    0.000    0.000    0.000    0.000 generic.py:6277(<genexpr>)
        2    0.000    0.000    0.000    0.000 socket.py:777(recv_multipart)
        8    0.000    0.000    0.000    0.000 managers.py:2017(_can_hold_na)
        8    0.000    0.000    0.000    0.000 common.py:633(is_integer_dtype)
        1    0.000    0.000    0.000    0.000 cast.py:879(infer_dtype_from_array)
        1    0.000    0.000    0.000    0.000 construction.py:547(_prep_ndarraylike)
        9    0.000    0.000    0.000    0.000 groupby.py:5565(_reindex_output)
      189    0.000    0.000    0.000    0.000 managers.py:196(blklocs)
        2    0.000    0.000    0.000    0.000 _strptime.py:392(_strptime)
        4    0.000    0.000    0.000    0.000 datetimelike.py:447(astype)
        9    0.000    0.000    0.000    0.000 fromnumeric.py:437(repeat)
      374    0.000    0.000    0.000    0.000 {function FrozenList.__getitem__ at 0x000002296AAA9120}
       27    0.000    0.000    0.000    0.000 base_events.py:738(time)
        4    0.000    0.000    0.000    0.000 base.py:3565(_wrap_intersection_result)
       10    0.000    0.000    0.000    0.000 api.py:383(<genexpr>)
      140    0.000    0.000    0.000    0.000 {method 'startswith' of 'str' objects}
       21    0.000    0.000    0.000    0.000 contextlib.py:104(__init__)
        4    0.000    0.000    0.000    0.000 accessors.py:59(__init__)
      144    0.000    0.000    0.000    0.000 multi.py:1046(nlevels)
       75    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
       52    0.000    0.000    0.000    0.000 {built-in method builtins.max}
      152    0.000    0.000    0.000    0.000 dtypes.py:531(validate_ordered)
       38    0.000    0.000    0.000    0.000 categorical.py:764(categories)
       12    0.000    0.000    0.000    0.000 _ufunc_config.py:426(__exit__)
        8    0.000    0.000    0.000    0.000 apply.py:1712(is_multi_agg_with_relabel)
       44    0.000    0.000    0.000    0.000 numerictypes.py:288(issubclass_)
        3    0.000    0.000    0.000    0.000 datetimes.py:2354(objects_to_datetime64)
        3    0.000    0.000    0.000    0.000 multi.py:1692(<genexpr>)
       10    0.000    0.000    0.000    0.000 series.py:978(__array__)
      133    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
       12    0.000    0.000    0.000    0.000 api.py:106(_get_distinct_objs)
        1    0.000    0.000    0.000    0.000 nanops.py:76(_f)
        8    0.000    0.000    0.000    0.000 merge.py:2680(_any)
      267    0.000    0.000    0.000    0.000 flags.py:55(allows_duplicate_labels)
       48    0.000    0.000    0.000    0.000 indexing.py:2738(is_label_like)
       28    0.000    0.000    0.000    0.000 groupby.py:1340(<genexpr>)
        1    0.000    0.000    0.000    0.000 multi.py:1966(_is_lexsorted)
        4    0.000    0.000    0.000    0.000 merge.py:958(_maybe_restore_index_levels)
       36    0.000    0.000    0.000    0.000 common.py:535(temp_setattr)
        4    0.000    0.000    0.000    0.000 _strptime.py:27(_getlang)
       79    0.000    0.000    0.000    0.000 construction.py:688(_sanitize_non_ordered)
       63    0.000    0.000    0.000    0.000 config.py:617(_select_options)
        5    0.000    0.000    0.000    0.000 indexing.py:2518(__getitem__)
        2    0.000    0.000    0.000    0.000 base.py:378(interleaved_dtype)
        4    0.000    0.000    0.000    0.000 encoder.py:183(encode)
        1    0.000    0.000    0.000    0.000 utils.py:186(parse_url_params)
       24    0.000    0.000    0.000    0.000 datetimelike.py:390(_get_getitem_freq)
       55    0.000    0.000    0.000    0.000 common.py:1122(<lambda>)
        1    0.000    0.000    0.000    0.000 construction.py:559(convert)
       72    0.000    0.000    0.000    0.000 utils.py:62(is_list_like_indexer)
        3    0.000    0.000    0.000    0.000 cast.py:1433(find_common_type)
        5    0.000    0.000    0.000    0.000 managers.py:2095(create_block_manager_from_blocks)
        1    0.000    0.000    0.000    0.000 multi.py:2002(_lexsort_depth)
        7    0.000    0.000    0.000    0.000 base.py:1730(_get_default_index_names)
       38    0.000    0.000    0.000    0.000 range.py:553(equals)
        6    0.000    0.000    0.000    0.000 {method 'strftime' of 'datetime.date' objects}
       22    0.000    0.000    0.000    0.000 generic.py:353(<genexpr>)
       56    0.000    0.000    0.000    0.000 indexing.py:992(_validate_key_length)
        9    0.000    0.000    0.000    0.000 base.py:448(size)
        6    0.000    0.000    0.000    0.000 events.py:111(__init__)
        2    0.000    0.000    0.000    0.000 base.py:5204(_get_join_target)
      175    0.000    0.000    0.000    0.000 managers.py:1940(_block)
       18    0.000    0.000    0.000    0.000 apply.py:653(<genexpr>)
        4    0.000    0.000    0.000    0.000 locale.py:582(getlocale)
       12    0.000    0.000    0.000    0.000 generic.py:3995(_maybe_update_cacher)
      165    0.000    0.000    0.000    0.000 generic.py:696(ndim)
        1    0.000    0.000    0.000    0.000 multi.py:4037(_lexsort_depth)
        3    0.000    0.000    0.000    0.000 traitlets.py:727(_cross_validate)
        1    0.000    0.000    0.000    0.000 cast.py:1605(maybe_cast_to_integer_array)
        8    0.000    0.000    0.000    0.000 common.py:195(any_not_none)
       90    0.000    0.000    0.000    0.000 indexing.py:981(<genexpr>)
       38    0.000    0.000    0.000    0.000 base.py:2220(_can_hold_na)
      276    0.000    0.000    0.000    0.000 base.py:363(ndim)
        2    0.000    0.000    0.000    0.000 numeric.py:143(ones)
       11    0.000    0.000    0.000    0.000 events.py:36(__init__)
       19    0.000    0.000    0.000    0.000 grouper.py:643(name)
        8    0.000    0.000    0.000    0.000 fromnumeric.py:1085(argsort)
        4    0.000    0.000    0.000    0.000 uuid.py:139(__init__)
       96    0.000    0.000    0.000    0.000 indexing.py:2772(<genexpr>)
       82    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 encoder.py:205(iterencode)
       14    0.000    0.000    0.000    0.000 {method 'reset' of '_contextvars.ContextVar' objects}
        6    0.000    0.000    0.000    0.000 enum.py:1556(__and__)
        3    0.000    0.000    0.000    0.000 _validators.py:354(validate_ascending)
       27    0.000    0.000    0.000    0.000 managers.py:2177(_grouping_func)
        2    0.000    0.000    0.000    0.000 base.py:1250(copy)
       88    0.000    0.000    0.000    0.000 common.py:126(_classes_and_not_datetimelike)
       36    0.000    0.000    0.000    0.000 managers.py:937(<genexpr>)
      121    0.000    0.000    0.000    0.000 multiarray.py:161(concatenate)
      122    0.000    0.000    0.000    0.000 managers.py:246(items)
       12    0.000    0.000    0.000    0.000 datetimelike.py:2152(unit)
        1    0.000    0.000    0.000    0.000 range.py:137(__new__)
        9    0.000    0.000    0.000    0.000 {method 'astype' of 'numpy.generic' objects}
       20    0.000    0.000    0.000    0.000 apply.py:1926(_managle_lambda_list)
        1    0.000    0.000    0.000    0.000 iostream.py:587(_schedule_in_thread)
        1    0.000    0.000    0.000    0.000 session.py:649(msg)
        7    0.000    0.000    0.000    0.000 series.py:789(values)
        4    0.000    0.000    0.000    0.000 base.py:675(empty)
        1    0.000    0.000    0.000    0.000 sql.py:2526(insert_statement)
       38    0.000    0.000    0.000    0.000 common.py:520(convert_to_list_like)
       43    0.000    0.000    0.000    0.000 indexing.py:161(iloc)
       63    0.000    0.000    0.000    0.000 config.py:649(_get_deprecated_option)
       43    0.000    0.000    0.000    0.000 {method 'transpose' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 zmqstream.py:710(_update_handler)
        1    0.000    0.000    0.000    0.000 session.py:675(sign)
        1    0.000    0.000    0.000    0.000 ioloop.py:596(call_later)
       50    0.000    0.000    0.000    0.000 {built-in method builtins.setattr}
        2    0.000    0.000    0.000    0.000 threading.py:1535(enumerate)
        8    0.000    0.000    0.000    0.000 enum.py:1545(__or__)
       24    0.000    0.000    0.000    0.000 enum.py:720(__call__)
        8    0.000    0.000    0.000    0.000 dispatch.py:17(should_extension_dispatch)
        1    0.000    0.000    0.000    0.000 parse.py:715(parse_qs)
        7    0.000    0.000    0.000    0.000 managers.py:2000(external_values)
        5    0.000    0.000    0.000    0.000 events.py:155(cancel)
       64    0.000    0.000    0.000    0.000 blocks.py:1249(shape)
       30    0.000    0.000    0.000    0.000 function.py:64(__call__)
        1    0.000    0.000    0.000    0.000 nanops.py:389(new_func)
        3    0.000    0.000    0.000    0.000 multi.py:1689(<genexpr>)
       42    0.000    0.000    0.000    0.000 enum.py:1538(_get_value)
       15    0.000    0.000    0.000    0.000 sql.py:2452(_get_unicode_name)
       15    0.000    0.000    0.000    0.000 datetimelike.py:2537(dtype_to_unit)
        8    0.000    0.000    0.000    0.000 base.py:4201(_convert_slice_indexer)
       64    0.000    0.000    0.000    0.000 indexing.py:1665(<genexpr>)
        8    0.000    0.000    0.000    0.000 array_ops.py:507(maybe_prepare_scalar_for_op)
       17    0.000    0.000    0.000    0.000 traitlets.py:676(__get__)
       42    0.000    0.000    0.000    0.000 base.py:692(_constructor)
       32    0.000    0.000    0.000    0.000 multi.py:595(<genexpr>)
        4    0.000    0.000    0.000    0.000 inspect.py:2743(__init__)
        5    0.000    0.000    0.000    0.000 construction.py:405(_check_values_indices_shape_match)
        3    0.000    0.000    0.000    0.000 _parser.py:1141(_to_decimal)
        2    0.000    0.000    0.000    0.000 __init__.py:164(match)
       14    0.000    0.000    0.000    0.000 base.py:3760(_assert_can_do_setop)
       22    0.000    0.000    0.000    0.000 extension.py:67(fget)
       17    0.000    0.000    0.000    0.000 threading.py:1155(_wait_for_tstate_lock)
       12    0.000    0.000    0.000    0.000 generic.py:4379(_set_is_copy)
        2    0.000    0.000    0.000    0.000 sql.py:2489(_register_date_adapters)
        4    0.000    0.000    0.000    0.000 sql.py:885(pandasSQL_builder)
       24    0.000    0.000    0.000    0.000 generic.py:6270(<genexpr>)
       20    0.000    0.000    0.000    0.000 grouper.py:983(is_in_axis)
       96    0.000    0.000    0.000    0.000 indexing.py:2781(<genexpr>)
        1    0.000    0.000    0.000    0.000 __init__.py:209(findall)
        8    0.000    0.000    0.000    0.000 shape_base.py:81(atleast_2d)
       46    0.000    0.000    0.000    0.000 {method 'as_arrays' of 'numpy._core._multiarray_umath._array_converter' objects}
        8    0.000    0.000    0.000    0.000 blocks.py:230(_can_hold_na)
       96    0.000    0.000    0.000    0.000 indexing.py:1180(<genexpr>)
        1    0.000    0.000    0.000    0.000 parse.py:757(parse_qsl)
        9    0.000    0.000    0.000    0.000 sorting.py:157(_int64_cut_off)
       31    0.000    0.000    0.000    0.000 grouper.py:947(<genexpr>)
       15    0.000    0.000    0.000    0.000 groupby.py:5607(<genexpr>)
        9    0.000    0.000    0.000    0.000 cast.py:331(maybe_downcast_numeric)
        1    0.000    0.000    0.000    0.000 traitlets.py:1512(_notify_trait)
        1    0.000    0.000    0.000    0.000 nanops.py:455(newfunc)
       10    0.000    0.000    0.000    0.000 datetimes.py:2514(_validate_dt64_dtype)
        8    0.000    0.000    0.000    0.000 missing.py:131(dispatch_fill_zeros)
        1    0.000    0.000    0.000    0.000 base.py:6219(_raise_if_missing)
       28    0.000    0.000    0.000    0.000 _validators.py:226(validate_bool_kwarg)
        3    0.000    0.000    0.000    0.000 multi.py:1657(_get_level_number)
       11    0.000    0.000    0.000    0.000 {method 'repeat' of 'numpy.ndarray' objects}
       46    0.000    0.000    0.000    0.000 {method 'wrap' of 'numpy._core._multiarray_umath._array_converter' objects}
        1    0.000    0.000    0.000    0.000 asyncio.py:210(call_at)
       20    0.000    0.000    0.000    0.000 grouper.py:638(_passed_categorical)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:448(end_date)
       24    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
        2    0.000    0.000    0.000    0.000 groupby.py:1129(__iter__)
        1    0.000    0.000    0.000    0.000 generic.py:2015(_wrap_agged_manager)
       10    0.000    0.000    0.000    0.000 datetimes.py:2310(_construct_from_dt64_naive)
        5    0.000    0.000    0.000    0.000 construction.py:585(_ensure_2d)
        1    0.000    0.000    0.000    0.000 nanops.py:604(nansum)
        2    0.000    0.000    0.000    0.000 multi.py:3845(_convert_can_do_setop)
        2    0.000    0.000    0.000    0.000 managers.py:958(fast_xs)
        9    0.000    0.000    0.000    0.000 ops.py:249(_get_output_shape)
        8    0.000    0.000    0.000    0.000 datetimelike.py:319(asi8)
       18    0.000    0.000    0.000    0.000 indexing.py:982(<genexpr>)
      190    0.000    0.000    0.000    0.000 dtypes.py:619(categories)
        9    0.000    0.000    0.000    0.000 ops.py:268(_get_out_dtype)
       52    0.000    0.000    0.000    0.000 {method 'lower' of 'str' objects}
        1    0.000    0.000    0.000    0.000 session.py:645(msg_header)
       11    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        2    0.000    0.000    0.000    0.000 traitlets.py:3474(validate)
        1    0.000    0.000    0.000    0.000 traitlets.py:1523(notify_change)
       16    0.000    0.000    0.000    0.000 series.py:865(_references)
      134    0.000    0.000    0.000    0.000 base.py:6672(_maybe_cast_indexer)
       45    0.000    0.000    0.000    0.000 {method 'item' of 'numpy.ndarray' objects}
       82    0.000    0.000    0.000    0.000 {built-in method numpy.lib.array_utils.normalize_axis_index}
       31    0.000    0.000    0.000    0.000 grouper.py:949(<genexpr>)
        4    0.000    0.000    0.000    0.000 merge.py:1554(_validate_left_right_on)
        5    0.000    0.000    0.000    0.000 base_events.py:446(create_future)
       20    0.000    0.000    0.000    0.000 construction.py:196(mgr_to_mgr)
       10    0.000    0.000    0.000    0.000 groupby.py:966(_selected_obj)
        2    0.000    0.000    0.000    0.000 frame.py:1096(_can_fast_transpose)
        7    0.000    0.000    0.000    0.000 blocks.py:249(external_values)
        6    0.000    0.000    0.000    0.000 typing.py:392(inner)
       62    0.000    0.000    0.000    0.000 blocks.py:266(mgr_locs)
        1    0.000    0.000    0.000    0.000 getlimits.py:688(__init__)
        2    0.000    0.000    0.000    0.000 iostream.py:710(_flush_buffers)
        4    0.000    0.000    0.000    0.000 fromnumeric.py:915(sort)
       59    0.000    0.000    0.000    0.000 datetimelike.py:2043(freq)
        2    0.000    0.000    0.000    0.000 generic.py:794(_set_axis_nocheck)
        1    0.000    0.000    0.000    0.000 base.py:5224(_from_join_target)
       12    0.000    0.000    0.000    0.000 base.py:166(__setattr__)
        1    0.000    0.000    0.000    0.000 {method 'findall' of 're.Pattern' objects}
        4    0.000    0.000    0.000    0.000 locale.py:464(_parse_localename)
       96    0.000    0.000    0.000    0.000 indexing.py:941(<genexpr>)
        2    0.000    0.000    0.000    0.000 base.py:7704(get_unanimous_names)
       12    0.000    0.000    0.000    0.000 {built-in method numpy._core._multiarray_umath._make_extobj}
       33    0.000    0.000    0.000    0.000 {method 'copy' of 'dict' objects}
       10    0.000    0.000    0.000    0.000 series.py:677(_constructor_expanddim)
        4    0.000    0.000    0.000    0.000 datetimes.py:638(__array__)
        2    0.000    0.000    0.000    0.000 jsonutil.py:107(json_default)
       50    0.000    0.000    0.000    0.000 {method 'add' of 'set' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.dir}
        1    0.000    0.000    0.000    0.000 traitlets.py:1527(_notify_observers)
       18    0.000    0.000    0.000    0.000 multi.py:1331(dtype)
       11    0.000    0.000    0.000    0.000 ops.py:578(__init__)
       36    0.000    0.000    0.000    0.000 {method 'endswith' of 'str' objects}
        7    0.000    0.000    0.000    0.000 sorting.py:546(ensure_key_mapped)
       20    0.000    0.000    0.000    0.000 grouper.py:999(is_in_obj)
        8    0.000    0.000    0.000    0.000 base.py:6016(argsort)
       21    0.000    0.000    0.000    0.000 inspect.py:302(isclass)
       20    0.000    0.000    0.000    0.000 apply.py:1733(<genexpr>)
        5    0.000    0.000    0.000    0.000 series.py:1220(_get_value)
        3    0.000    0.000    0.000    0.000 _parser.py:428(append)
        2    0.000    0.000    0.000    0.000 typing.py:1221(__instancecheck__)
       13    0.000    0.000    0.000    0.000 selector_events.py:750(_process_events)
       11    0.000    0.000    0.000    0.000 indexing.py:2721(is_nested_tuple)
       12    0.000    0.000    0.000    0.000 base.py:332(array)
       25    0.000    0.000    0.000    0.000 numba_.py:18(maybe_use_numba)
        3    0.000    0.000    0.000    0.000 common.py:389(standardize_mapping)
        2    0.000    0.000    0.000    0.000 _dtype.py:184(_datetime_metadata_str)
        2    0.000    0.000    0.000    0.000 _validators.py:141(validate_kwargs)
       82    0.000    0.000    0.000    0.000 {built-in method _operator.index}
        8    0.000    0.000    0.000    0.000 series.py:6137(_align_for_op)
        4    0.000    0.000    0.000    0.000 uuid.py:280(__str__)
       12    0.000    0.000    0.000    0.000 <string>:2(__init__)
       12    0.000    0.000    0.000    0.000 datetimes.py:103(tz_to_dtype)
       24    0.000    0.000    0.000    0.000 _parser.py:208(isnum)
       47    0.000    0.000    0.000    0.000 cast.py:1763(np_can_hold_element)
        4    0.000    0.000    0.000    0.000 {built-in method _locale.setlocale}
       14    0.000    0.000    0.000    0.000 {method 'set' of '_contextvars.ContextVar' objects}
       20    0.000    0.000    0.000    0.000 datetimes.py:576(tz)
        1    0.000    0.000    0.000    0.000 inspect.py:3029(__init__)
        1    0.000    0.000    0.000    0.000 _parser.py:221(__init__)
       14    0.000    0.000    0.000    0.000 parse.py:793(_unquote)
        4    0.000    0.000    0.000    0.000 {built-in method nt.urandom}
      114    0.000    0.000    0.000    0.000 dtypes.py:632(ordered)
       31    0.000    0.000    0.000    0.000 grouper.py:948(<genexpr>)
        2    0.000    0.000    0.000    0.000 _parser.py:1006(_find_hms_idx)
       20    0.000    0.000    0.000    0.000 {method 'upper' of 'str' objects}
       20    0.000    0.000    0.000    0.000 grouper.py:1080(_is_label_like)
       20    0.000    0.000    0.000    0.000 managers.py:241(is_single_block)
        7    0.000    0.000    0.000    0.000 blocks.py:2827(external_values)
       36    0.000    0.000    0.000    0.000 generic.py:1793(<genexpr>)
       41    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
       16    0.000    0.000    0.000    0.000 common.py:295(maybe_make_list)
        9    0.000    0.000    0.000    0.000 ops.py:512(_validate_axis)
       90    0.000    0.000    0.000    0.000 ops.py:592(groupings)
       38    0.000    0.000    0.000    0.000 categorical.py:494(dtype)
       39    0.000    0.000    0.000    0.000 {built-in method sys.getrefcount}
        6    0.000    0.000    0.000    0.000 _mixins.py:186(_from_factorized)
       27    0.000    0.000    0.000    0.000 {built-in method time.monotonic}
       31    0.000    0.000    0.000    0.000 {method 'count' of 'list' objects}
        4    0.000    0.000    0.000    0.000 base.py:756(_shallow_copy)
       59    0.000    0.000    0.000    0.000 missing.py:1073(clean_reindex_fill_method)
       14    0.000    0.000    0.000    0.000 sorting.py:238(is_int64_overflow_possible)
       17    0.000    0.000    0.000    0.000 traitlets.py:629(get)
       82    0.000    0.000    0.000    0.000 fromnumeric.py:1842(_ravel_dispatcher)
        1    0.000    0.000    0.000    0.000 hmac.py:122(copy)
        8    0.000    0.000    0.000    0.000 shape_base.py:215(_vhstack_dispatcher)
        2    0.000    0.000    0.000    0.000 typing.py:1492(__subclasscheck__)
        2    0.000    0.000    0.000    0.000 multi.py:3804(_get_reconciled_name_object)
        1    0.000    0.000    0.000    0.000 _parser.py:62(__init__)
        1    0.000    0.000    0.000    0.000 indexing.py:782(_maybe_mask_setitem_value)
        4    0.000    0.000    0.000    0.000 locale.py:381(normalize)
       23    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
        4    0.000    0.000    0.000    0.000 merge.py:1255(<lambda>)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:420(hourly)
       11    0.000    0.000    0.000    0.000 _parser.py:203(isword)
       14    0.000    0.000    0.000    0.000 parse.py:830(unquote_plus)
        4    0.000    0.000    0.000    0.000 datetimelike.py:354(__array__)
        2    0.000    0.000    0.000    0.000 common.py:209(count_not_none)
       22    0.000    0.000    0.000    0.000 generic.py:349(<genexpr>)
       16    0.000    0.000    0.000    0.000 apply.py:671(<genexpr>)
       38    0.000    0.000    0.000    0.000 categorical.py:2139(_codes)
        1    0.000    0.000    0.000    0.000 ops.py:1191(_get_splitter)
       12    0.000    0.000    0.000    0.000 concat.py:693(_get_result_dim)
        9    0.000    0.000    0.000    0.000 apply.py:121(__init__)
        2    0.000    0.000    0.000    0.000 base.py:2155(<genexpr>)
       24    0.000    0.000    0.000    0.000 enum.py:1123(__new__)
       82    0.000    0.000    0.000    0.000 _function_base_impl.py:5701(_append_dispatcher)
       16    0.000    0.000    0.000    0.000 {method 'append' of 'collections.deque' objects}
        5    0.000    0.000    0.000    0.000 {method 'cursor' of 'sqlite3.Connection' objects}
       23    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 datetimes.py:2623(_infer_tz_from_endpoints)
        1    0.000    0.000    0.000    0.000 session.py:272(msg_header)
       16    0.000    0.000    0.000    0.000 multi.py:1653(inferred_type)
       14    0.000    0.000    0.000    0.000 threading.py:1198(ident)
        1    0.000    0.000    0.000    0.000 base.py:5425(_getitem_slice)
       20    0.000    0.000    0.000    0.000 common.py:1194(<lambda>)
       22    0.000    0.000    0.000    0.000 generic.py:364(<genexpr>)
        1    0.000    0.000    0.000    0.000 multi.py:1062(levshape)
        1    0.000    0.000    0.000    0.000 merge.py:2588(_convert_arrays_and_get_rizer_klass)
       10    0.000    0.000    0.000    0.000 {built-in method builtins.min}
        6    0.000    0.000    0.000    0.000 {built-in method _heapq.heappop}
       12    0.000    0.000    0.000    0.000 <string>:2(__hash__)
       15    0.000    0.000    0.000    0.000 managers.py:291(arrays)
        1    0.000    0.000    0.000    0.000 history.py:820(_writeout_output_cache)
       16    0.000    0.000    0.000    0.000 common.py:199(<genexpr>)
        2    0.000    0.000    0.000    0.000 zmqstream.py:566(sending)
        2    0.000    0.000    0.000    0.000 {method 'isoformat' of 'datetime.datetime' objects}
        1    0.000    0.000    0.000    0.000 reshape.py:363(_repeater)
        9    0.000    0.000    0.000    0.000 ops.py:212(_get_cython_vals)
        5    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        8    0.000    0.000    0.000    0.000 utils.py:38(is_valid_positional_slice)
        2    0.000    0.000    0.000    0.000 traitlets.py:3624(validate_elements)
       20    0.000    0.000    0.000    0.000 datetimes.py:2570(_validate_tz_from_dtype)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:437(start_date)
        1    0.000    0.000    0.000    0.000 parse.py:374(urlparse)
        2    0.000    0.000    0.000    0.000 cast.py:1392(np_find_common_type)
        1    0.000    0.000    0.000    0.000 series.py:1484(_check_is_chained_assignment_possible)
        9    0.000    0.000    0.000    0.000 ops.py:696(names)
       30    0.000    0.000    0.000    0.000 {built-in method builtins.id}
        8    0.000    0.000    0.000    0.000 datetimelike.py:2148(_creso)
       29    0.000    0.000    0.000    0.000 base.py:1176(_maybe_disallow_fill)
        6    0.000    0.000    0.000    0.000 utils.py:123(check_setitem_lengths)
        2    0.000    0.000    0.000    0.000 inference.py:373(is_sequence)
        1    0.000    0.000    0.000    0.000 {method 'copy' of '_hashlib.HMAC' objects}
       56    0.000    0.000    0.000    0.000 series.py:1480(_clear_item_cache)
        1    0.000    0.000    0.000    0.000 inference.py:404(is_dataclass)
        2    0.000    0.000    0.000    0.000 merge.py:2333(_get_empty_indexer)
        8    0.000    0.000    0.000    0.000 core.py:160(<genexpr>)
        5    0.000    0.000    0.000    0.000 indexing.py:2603(_tuplify)
        4    0.000    0.000    0.000    0.000 _ufunc_config.py:458(_no_nep50_warning)
       43    0.000    0.000    0.000    0.000 multiarray.py:901(bincount)
        7    0.000    0.000    0.000    0.000 indexing.py:1165(_check_deprecated_callable_usage)
        8    0.000    0.000    0.000    0.000 shape_base.py:207(_arrays_for_stack_dispatcher)
       15    0.000    0.000    0.000    0.000 {method 'popleft' of 'collections.deque' objects}
        2    0.000    0.000    0.000    0.000 frame.py:1111(_values)
       20    0.000    0.000    0.000    0.000 {method 'read' of '_io.StringIO' objects}
        1    0.000    0.000    0.000    0.000 _parser.py:474(resolve_ymd)
        2    0.000    0.000    0.000    0.000 sql.py:1211(_index_name)
       20    0.000    0.000    0.000    0.000 {method 'encode' of 'str' objects}
        6    0.000    0.000    0.000    0.000 common.py:641(fill_missing_names)
        1    0.000    0.000    0.000    0.000 _parser.py:395(__init__)
        5    0.000    0.000    0.000    0.000 indexing.py:305(loc)
       10    0.000    0.000    0.000    0.000 numeric.py:2399(<genexpr>)
        2    0.000    0.000    0.000    0.000 multi.py:3816(_maybe_match_names)
        4    0.000    0.000    0.000    0.000 merge.py:1256(<lambda>)
        3    0.000    0.000    0.000    0.000 _parser.py:336(hms)
       41    0.000    0.000    0.000    0.000 _function_base_impl.py:5514(_insert_dispatcher)
        8    0.000    0.000    0.000    0.000 base.py:915(__array__)
        2    0.000    0.000    0.000    0.000 core.py:254(_write_to_db)
       14    0.000    0.000    0.000    0.000 base.py:3211(_validate_sort_keyword)
        2    0.000    0.000    0.000    0.000 core.py:335(<lambda>)
        6    0.000    0.000    0.000    0.000 {built-in method _sqlite3.register_adapter}
        2    0.000    0.000    0.000    0.000 queue.py:97(empty)
        1    0.000    0.000    0.000    0.000 hmac.py:161(hexdigest)
        4    0.000    0.000    0.000    0.000 nanops.py:79(<genexpr>)
        6    0.000    0.000    0.000    0.000 generic.py:580(_get_axis_name)
       35    0.000    0.000    0.000    0.000 numeric.py:2426(_array_equal_dispatcher)
        6    0.000    0.000    0.000    0.000 {built-in method _heapq.heappush}
       31    0.000    0.000    0.000    0.000 {method 'replace' of 'str' objects}
       41    0.000    0.000    0.000    0.000 numeric.py:1433(_moveaxis_dispatcher)
        4    0.000    0.000    0.000    0.000 typing.py:1285(__hash__)
        6    0.000    0.000    0.000    0.000 blocks.py:259(_standardize_fill_value)
       15    0.000    0.000    0.000    0.000 {method 'find' of 'str' objects}
       10    0.000    0.000    0.000    0.000 datetimelike.py:2063(_maybe_pin_freq)
        2    0.000    0.000    0.000    0.000 _validators.py:398(validate_inclusive)
        2    0.000    0.000    0.000    0.000 iostream.py:505(parent_header)
       28    0.000    0.000    0.000    0.000 {method 'isdigit' of 'str' objects}
       20    0.000    0.000    0.000    0.000 grouper.py:661(_ilevel)
        9    0.000    0.000    0.000    0.000 ops.py:284(_get_result_dtype)
        1    0.000    0.000    0.000    0.000 session.py:600(msg_id)
        6    0.000    0.000    0.000    0.000 base.py:7719(<genexpr>)
        5    0.000    0.000    0.000    0.000 events.py:72(cancel)
        1    0.000    0.000    0.000    0.000 session.py:198(utcnow)
       22    0.000    0.000    0.000    0.000 series.py:660(_constructor)
        1    0.000    0.000    0.000    0.000 inspect.py:176(get_annotations)
        1    0.000    0.000    0.000    0.000 ops.py:639(group_keys_seq)
        4    0.000    0.000    0.000    0.000 blocks.py:2586(get_values)
       32    0.000    0.000    0.000    0.000 contextlib.py:440(__init__)
       32    0.000    0.000    0.000    0.000 contextlib.py:446(__exit__)
        1    0.000    0.000    0.000    0.000 iostream.py:718(_rotate_buffers)
        2    0.000    0.000    0.000    0.000 common.py:505(get_rename_function)
       10    0.000    0.000    0.000    0.000 indexing.py:2593(_convert_key)
        3    0.000    0.000    0.000    0.000 pivot.py:173(<genexpr>)
        4    0.000    0.000    0.000    0.000 hmac.py:117(update)
       12    0.000    0.000    0.000    0.000 _ufunc_config.py:400(__init__)
       10    0.000    0.000    0.000    0.000 indexing.py:676(iat)
       18    0.000    0.000    0.000    0.000 _shape_base_impl.py:1269(<genexpr>)
       11    0.000    0.000    0.000    0.000 {method '__exit__' of '_thread.lock' objects}
        5    0.000    0.000    0.000    0.000 _parser.py:213(isspace)
        6    0.000    0.000    0.000    0.000 {built-in method _contextvars.copy_context}
       22    0.000    0.000    0.000    0.000 generic.py:374(<genexpr>)
       18    0.000    0.000    0.000    0.000 grouper.py:722(_result_index)
        8    0.000    0.000    0.000    0.000 merge.py:1012(<genexpr>)
        9    0.000    0.000    0.000    0.000 groupby.py:1567(_maybe_transpose_result)
        2    0.000    0.000    0.000    0.000 base.py:5088(values)
        4    0.000    0.000    0.000    0.000 merge.py:2674(_should_fill)
        4    0.000    0.000    0.000    0.000 {built-in method _sqlite3.register_converter}
        5    0.000    0.000    0.000    0.000 {built-in method _abc._abc_subclasscheck}
        4    0.000    0.000    0.000    0.000 common.py:174(not_none)
       11    0.000    0.000    0.000    0.000 {method 'isalpha' of 'str' objects}
        4    0.000    0.000    0.000    0.000 common.py:626(get_cython_func)
        5    0.000    0.000    0.000    0.000 inspect.py:3076(<genexpr>)
       18    0.000    0.000    0.000    0.000 datetimes.py:217(_scalar_type)
        2    0.000    0.000    0.000    0.000 astype.py:249(astype_is_view)
        1    0.000    0.000    0.000    0.000 {method 'hexdigest' of '_hashlib.HMAC' objects}
        2    0.000    0.000    0.000    0.000 traitlets.py:2304(validate)
        8    0.000    0.000    0.000    0.000 array_ops.py:594(_bool_arith_check)
        2    0.000    0.000    0.000    0.000 reshape.py:196(<genexpr>)
        1    0.000    0.000    0.000    0.000 {built-in method _thread.allocate_lock}
        8    0.000    0.000    0.000    0.000 {method 'update' of 'dict' objects}
       14    0.000    0.000    0.000    0.000 parse.py:692(unquote)
       10    0.000    0.000    0.000    0.000 {built-in method builtins.iter}
       17    0.000    0.000    0.000    0.000 threading.py:601(is_set)
        2    0.000    0.000    0.000    0.000 iostream.py:550(_is_master_process)
        2    0.000    0.000    0.000    0.000 api.py:330(_sanitize_and_check)
       15    0.000    0.000    0.000    0.000 {method 'decode' of 'bytes' objects}
        4    0.000    0.000    0.000    0.000 {method 'sort' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 traitlets.py:2635(validate)
        2    0.000    0.000    0.000    0.000 ioloop.py:541(time)
        5    0.000    0.000    0.000    0.000 sql.py:1118(<genexpr>)
        1    0.000    0.000    0.000    0.000 _parser.py:1099(_parse_min_sec)
        2    0.000    0.000    0.000    0.000 _validators.py:128(_check_for_invalid_keys)
        6    0.000    0.000    0.000    0.000 _mixins.py:194(_values_for_factorize)
        3    0.000    0.000    0.000    0.000 {method 'index' of 'list' objects}
       32    0.000    0.000    0.000    0.000 contextlib.py:443(__enter__)
       16    0.000    0.000    0.000    0.000 multiarray.py:1140(putmask)
        1    0.000    0.000    0.000    0.000 __init__.py:535(_check_cacher)
        1    0.000    0.000    0.000    0.000 <string>:1(<lambda>)
        4    0.000    0.000    0.000    0.000 blocks.py:320(__len__)
       16    0.000    0.000    0.000    0.000 concat.py:807(<genexpr>)
        2    0.000    0.000    0.000    0.000 base.py:798(tolist)
        2    0.000    0.000    0.000    0.000 base.py:397(ensure_np_dtype)
       21    0.000    0.000    0.000    0.000 base_events.py:2017(get_debug)
        3    0.000    0.000    0.000    0.000 multi.py:1076(<genexpr>)
       12    0.000    0.000    0.000    0.000 concat.py:770(_maybe_check_integrity)
        7    0.000    0.000    0.000    0.000 common.py:686(<lambda>)
        1    0.000    0.000    0.000    0.000 dataclasses.py:1301(is_dataclass)
        1    0.000    0.000    0.000    0.000 base.py:4436(_wrap_reindex_result)
        5    0.000    0.000    0.000    0.000 {built-in method _asyncio.get_running_loop}
       11    0.000    0.000    0.000    0.000 base_events.py:543(_check_closed)
        1    0.000    0.000    0.000    0.000 generic.py:4585(_is_view)
        3    0.000    0.000    0.000    0.000 reshape.py:168(<genexpr>)
       10    0.000    0.000    0.000    0.000 {method 'pop' of 'list' objects}
        3    0.000    0.000    0.000    0.000 nanops.py:72(check)
        4    0.000    0.000    0.000    0.000 managers.py:1128(value_getitem)
        1    0.000    0.000    0.000    0.000 {built-in method now}
        2    0.000    0.000    0.000    0.000 _parser.py:319(jump)
        5    0.000    0.000    0.000    0.000 {method 'cancelled' of '_asyncio.Future' objects}
        1    0.000    0.000    0.000    0.000 _parser.py:380(validate)
        1    0.000    0.000    0.000    0.000 threading.py:299(__enter__)
        2    0.000    0.000    0.000    0.000 _validators.py:49(_check_for_default_values)
       13    0.000    0.000    0.000    0.000 ops.py:125(__init__)
        1    0.000    0.000    0.000    0.000 session.py:281(extract_header)
        1    0.000    0.000    0.000    0.000 construction.py:532(treat_as_nested)
        1    0.000    0.000    0.000    0.000 {method 'values' of 'mappingproxy' objects}
        4    0.000    0.000    0.000    0.000 {method 'update' of '_hashlib.HMAC' objects}
        2    0.000    0.000    0.000    0.000 _parser.py:329(month)
        2    0.000    0.000    0.000    0.000 base.py:5058(_can_use_libjoin)
        1    0.000    0.000    0.000    0.000 threading.py:314(_is_owned)
        1    0.000    0.000    0.000    0.000 indexing.py:829(_ensure_listlike_indexer)
       12    0.000    0.000    0.000    0.000 fromnumeric.py:3063(_prod_dispatcher)
        1    0.000    0.000    0.000    0.000 ops.py:1129(__init__)
        4    0.000    0.000    0.000    0.000 {built-in method from_bytes}
        6    0.000    0.000    0.000    0.000 fromnumeric.py:2508(_all_dispatcher)
       12    0.000    0.000    0.000    0.000 common.py:178(<genexpr>)
        1    0.000    0.000    0.000    0.000 ops.py:692(levels)
        1    0.000    0.000    0.000    0.000 base.py:4440(_maybe_preserve_names)
        6    0.000    0.000    0.000    0.000 {built-in method nt.getpid}
        1    0.000    0.000    0.000    0.000 nanops.py:253(_get_values)
        4    0.000    0.000    0.000    0.000 base.py:159(_freeze)
        3    0.000    0.000    0.000    0.000 {method 'close' of 'sqlite3.Cursor' objects}
        1    0.000    0.000    0.000    0.000 iostream.py:725(_hooks)
        2    0.000    0.000    0.000    0.000 iostream.py:138(_event_pipe)
       16    0.000    0.000    0.000    0.000 common.py:511(f)
        1    0.000    0.000    0.000    0.000 managers.py:480(is_view)
        5    0.000    0.000    0.000    0.000 {method '__exit__' of '_thread.RLock' objects}
        1    0.000    0.000    0.000    0.000 threading.py:308(_release_save)
       11    0.000    0.000    0.000    0.000 managers.py:1828(ndim)
        2    0.000    0.000    0.000    0.000 base_events.py:1907(_add_callback)
        4    0.000    0.000    0.000    0.000 multiarray.py:1327(shares_memory)
        9    0.000    0.000    0.000    0.000 datetimes.py:537(_check_compatible_with)
        2    0.000    0.000    0.000    0.000 {method 'groupdict' of 're.Match' objects}
        6    0.000    0.000    0.000    0.000 cast.py:1463(<genexpr>)
       10    0.000    0.000    0.000    0.000 common.py:213(<genexpr>)
        2    0.000    0.000    0.000    0.000 traitlets.py:3486(validate_elements)
        2    0.000    0.000    0.000    0.000 multiarray.py:436(lexsort)
        9    0.000    0.000    0.000    0.000 base.py:1979(nlevels)
        2    0.000    0.000    0.000    0.000 {built-in method fromkeys}
        5    0.000    0.000    0.000    0.000 {method 'isspace' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.locals}
        4    0.000    0.000    0.000    0.000 cast.py:1472(<genexpr>)
        4    0.000    0.000    0.000    0.000 ops.py:161(get_kind_from_how)
        4    0.000    0.000    0.000    0.000 {method 'isidentifier' of 'str' objects}
        8    0.000    0.000    0.000    0.000 fromnumeric.py:2605(_cumsum_dispatcher)
        2    0.000    0.000    0.000    0.000 _validators.py:450(check_dtype_backend)
       11    0.000    0.000    0.000    0.000 generic.py:4518(_check_inplace_and_allows_duplicate_labels)
        2    0.000    0.000    0.000    0.000 queue.py:209(_qsize)
        9    0.000    0.000    0.000    0.000 fromnumeric.py:433(_repeat_dispatcher)
        2    0.000    0.000    0.000    0.000 sql.py:94(_process_parse_dates_argument)
        8    0.000    0.000    0.000    0.000 inspect.py:2796(name)
        1    0.000    0.000    0.000    0.000 iostream.py:216(_check_mp_mode)
        4    0.000    0.000    0.000    0.000 sql.py:2653(__init__)
        4    0.000    0.000    0.000    0.000 sql.py:1451(__exit__)
        4    0.000    0.000    0.000    0.000 _shape_base_impl.py:1279(<genexpr>)
        7    0.000    0.000    0.000    0.000 {method 'partition' of 'str' objects}
        1    0.000    0.000    0.000    0.000 _parser.py:342(ampm)
        5    0.000    0.000    0.000    0.000 fromnumeric.py:2776(_max_dispatcher)
        9    0.000    0.000    0.000    0.000 _shape_base_impl.py:1193(_tile_dispatcher)
        4    0.000    0.000    0.000    0.000 {method '__contains__' of 'frozenset' objects}
        3    0.000    0.000    0.000    0.000 {method 'is_finite' of 'decimal.Decimal' objects}
        4    0.000    0.000    0.000    0.000 encoder.py:105(__init__)
        7    0.000    0.000    0.000    0.000 inspect.py:2800(default)
        1    0.000    0.000    0.000    0.000 _parser.py:322(weekday)
        2    0.000    0.000    0.000    0.000 pivot.py:493(_convert_by)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:173(wind_speed_unit)
     19/2    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.000    0.000 function.py:413(validate_func)
        1    0.000    0.000    0.000    0.000 {method 'swapaxes' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 range.py:463(__iter__)
        5    0.000    0.000    0.000    0.000 _function_base_impl.py:5331(_delete_dispatcher)
        2    0.000    0.000    0.000    0.000 {method 'tolist' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 parse.py:119(_coerce_args)
        5    0.000    0.000    0.000    0.000 base_events.py:1917(_timer_handle_cancelled)
        7    0.000    0.000    0.000    0.000 multiarray.py:83(empty_like)
        1    0.000    0.000    0.000    0.000 nanops.py:324(_get_dtype_max)
        2    0.000    0.000    0.000    0.000 selectors.py:275(_key_from_fd)
        7    0.000    0.000    0.000    0.000 inspect.py:2804(annotation)
        2    0.000    0.000    0.000    0.000 jsonutil.py:38(_ensure_tzinfo)
        6    0.000    0.000    0.000    0.000 blocks.py:805(_maybe_copy)
        4    0.000    0.000    0.000    0.000 fromnumeric.py:911(_sort_dispatcher)
        2    0.000    0.000    0.000    0.000 inspect.py:378(isfunction)
        1    0.000    0.000    0.000    0.000 _parser.py:1133(_parsems)
        2    0.000    0.000    0.000    0.000 construction.py:368(<genexpr>)
        4    0.000    0.000    0.000    0.000 {method 'pop' of 'set' objects}
        2    0.000    0.000    0.000    0.000 {method 'get' of '_contextvars.ContextVar' objects}
        2    0.000    0.000    0.000    0.000 {built-in method time.time}
        8    0.000    0.000    0.000    0.000 fromnumeric.py:1081(_argsort_dispatcher)
        1    0.000    0.000    0.000    0.000 getlimits.py:712(max)
        1    0.000    0.000    0.000    0.000 blocks.py:209(is_object)
        4    0.000    0.000    0.000    0.000 sql.py:1448(__enter__)
        4    0.000    0.000    0.000    0.000 {method 'toordinal' of 'datetime.date' objects}
        4    0.000    0.000    0.000    0.000 inspect.py:2808(kind)
        2    0.000    0.000    0.000    0.000 tz.py:74(utcoffset)
        4    0.000    0.000    0.000    0.000 {built-in method builtins.abs}
        1    0.000    0.000    0.000    0.000 events.py:127(__lt__)
        8    0.000    0.000    0.000    0.000 shape_base.py:77(_atleast_2d_dispatcher)
        4    0.000    0.000    0.000    0.000 cast.py:1474(<genexpr>)
        4    0.000    0.000    0.000    0.000 merge.py:815(_maybe_require_matching_dtypes)
        6    0.000    0.000    0.000    0.000 cast.py:1479(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:7652(ensure_has_len)
        1    0.000    0.000    0.000    0.000 range.py:216(_validate_dtype)
        1    0.000    0.000    0.000    0.000 {method '__enter__' of '_thread.lock' objects}
        2    0.000    0.000    0.000    0.000 {method 'write' of '_io.StringIO' objects}
        4    0.000    0.000    0.000    0.000 zmqstream.py:562(receiving)
        2    0.000    0.000    0.000    0.000 numeric.py:2311(_isclose_dispatcher)
        1    0.000    0.000    0.000    0.000 iostream.py:213(_is_master_process)
        2    0.000    0.000    0.000    0.000 frame.py:655(_constructor)
        2    0.000    0.000    0.000    0.000 multiarray.py:361(where)
        2    0.000    0.000    0.000    0.000 datetimelike.py:2474(validate_periods)
        1    0.000    0.000    0.000    0.000 blocks.py:2577(is_view)
        1    0.000    0.000    0.000    0.000 base.py:7683(_validate_join_method)
        3    0.000    0.000    0.000    0.000 range.py:315(step)
        1    0.000    0.000    0.000    0.000 {method 'getvalue' of '_io.StringIO' objects}
        1    0.000    0.000    0.000    0.000 _parser.py:402(has_year)
        1    0.000    0.000    0.000    0.000 indexing.py:2684(convert_missing_indexer)
        2    0.000    0.000    0.000    0.000 range.py:231(_constructor)
        1    0.000    0.000    0.000    0.000 _parser.py:360(convertyear)
        1    0.000    0.000    0.000    0.000 nanops.py:209(_maybe_get_mask)
        2    0.000    0.000    0.000    0.000 datetimes.py:2666(_maybe_normalize_endpoints)
        3    0.000    0.000    0.000    0.000 range.py:280(start)
        4    0.000    0.000    0.000    0.000 merge.py:821(_validate_tolerance)
        2    0.000    0.000    0.000    0.000 {method 'weekday' of 'datetime.date' objects}
        2    0.000    0.000    0.000    0.000 abstract_classes.py:143(longitude)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:429(daily)
        2    0.000    0.000    0.000    0.000 {method 'end' of 're.Match' objects}
        2    0.000    0.000    0.000    0.000 inspect.py:3093(return_annotation)
        1    0.000    0.000    0.000    0.000 generic.py:4387(_check_is_chained_assignment_possible)
        2    0.000    0.000    0.000    0.000 range.py:298(stop)
        1    0.000    0.000    0.000    0.000 {method 'close' of '_io.StringIO' objects}
        1    0.000    0.000    0.000    0.000 _parser.py:1056(_could_be_tzname)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:155(elevation)
        1    0.000    0.000    0.000    0.000 {method 'release' of '_thread.lock' objects}
        1    0.000    0.000    0.000    0.000 abstract_classes.py:218(apikey)
        1    0.000    0.000    0.000    0.000 _parser.py:186(__iter__)
        1    0.000    0.000    0.000    0.000 nanops.py:1499(_maybe_null_out)
        2    0.000    0.000    0.000    0.000 multiarray.py:678(result_type)
        1    0.000    0.000    0.000    0.000 managers.py:1989(index)
        1    0.000    0.000    0.000    0.000 traitlets.py:2558(_validate_bounds)
        1    0.000    0.000    0.000    0.000 inspect.py:3089(parameters)
        1    0.000    0.000    0.000    0.000 iostream.py:255(closed)
        1    0.000    0.000    0.000    0.000 indexing.py:1248(_has_valid_setitem_indexer)
        2    0.000    0.000    0.000    0.000 abstract_classes.py:132(latitude)
        1    0.000    0.000    0.000    0.000 hmac.py:139(_current)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:182(precipitation_unit)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:191(timeformat)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:200(timezone)
        1    0.000    0.000    0.000    0.000 abstract_classes.py:164(temperature_unit)
        1    0.000    0.000    0.000    0.000 parse.py:108(_noop)'''

In [67]:
import re
cols = out.split('\n')[1]
cols = cols.strip()
cols = re.split(r'\s+', cols)

data = out.split('\n')[2:]
data = [re.split(r'\s+', i.strip(),5) for i in data]

a = pd.DataFrame(data, columns=cols)


In [83]:
a.tottime = a.tottime.astype(float)
a.percall = a.percall.astype(float)
a.cumtime = a.cumtime.astype(float)

,ncalls,tottime,percall,cumtime,percall,filename:lineno(function)
0,12,0.000,0.000,43.712,3.643,base_events.py:1922(_run_once)
1,12,0.040,0.003,43.709,3.642,selectors.py:319(select)
2,12,0.055,0.005,41.320,3.443,selectors.py:313(_select)
3,10,15.434,1.543,15.434,1.543,{method 'execute' of 'sqlite3.Connection' obje...
4,12,12.150,1.013,13.897,1.158,{built-in method select.select}
...,...,...,...,...,...,...
1407,1,0.000,0.000,0.000,0.000,abstract_classes.py:182(precipitation_unit)
1408,1,0.000,0.000,0.000,0.000,abstract_classes.py:191(timeformat)
1409,1,0.000,0.000,0.000,0.000,abstract_classes.py:200(timezone)
1410,1,0.000,0.000,0.000,0.000,abstract_classes.py:164(temperature_unit)
